<a href="https://colab.research.google.com/github/jacobgreen4477/The-4th-ETRI-AI-Human-Understanding-Competition/blob/main/etri_baseline_v4_0_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> title : 제 4회 ETRI 휴먼이해 인공지능 논문경진대회 <br>
> author : hjy <br>

In our study, we used smartphones, smartwatches, sleep sensors, and self-recording apps to collect daily life logs and sleep health records of study participants in 2024.The data collection procedures and methods followed a similar approach to those used in previous studies. Here, we pu﻿blicly provide the following 12 data items, which comprise a total of 700 days' worth of lifelog data, strictly for non-commercial and academic research purposes only.
- mACStatus: Indicates whether the smartphone is currently being charged.
- mActivity: Value calculated by the Google Activity Recognition API.
- mAmbience: Ambient sound identification labels and their respective probabilities.
- mBle: Bluetooth devices around individual subject.
- mGps: Multiple GPS coordinates measured within a single minute using the smartphone.
- mLight: Ambient light measured by the smartphone.
- mScreenStatus: Indicates whether the smartphone screen is in use.
- mUsageStats: Indicates which apps were used on the smartphone and for how long.
- mWifi: Wifi devices around individual subject.
- wHr: Heart rate readings recorded by the smartwatch.
- wLight: Ambient light measured by the smartwatch.
- wPedo: Step data recorded by the smartwatch.

For the purpose of training a learning model to predict sleep health, fatigue, and stress, the following six metrics were derived from sleep sensor data and self-reported survey records. Each metric consists of values categorized into either two levels (0, 1) or three levels (0, 1, 2), depending on the specific metric. The detailed classification criteria for each metric's levels will be provided in a separate document.These
metrics assign a value of 0 for sleep records that do not meet the recommended guidelines.For instance, the first questionnaire metric (Q1) is assigned a value of 1 on days when an
individual’s self-reported sleep quality exceeds their average over the experimental period, and 0 when it
falls below that average. Similarly, the second and third metrics (Q2 and Q3) are assigned a value of 0
on days when the participant’s fatigue and stress levels, respectively, exceed their average, and a value of
1 when these levels are below average.

- Q1: Overall sleep quality as perceived by a subject immediately after waking up.
- Q2: Physical fatigue of a subject just before sleep.
- Q3: Stress level experienced by a subject just before sleep.
- S1: Adherence to sleep guidelines for total sleep time (TST).
- S2: Adherence to sleep guidelines for sleep efficiency (SE).
- S3: Adherence to sleep guidelines for sleep onset latency (SOL, or SL).

수면 건강, 피로, 스트레스 예측을 위한 학습 모델을 훈련시키기 위해, 수면 센서 데이터와 자기 보고식 설문 기록을 기반으로 다음의 6가지 지표를 도출했습니다.
각 지표는 해당 항목에 따라 두 수준(0, 1) 또는 세 수준(0, 1, 2)으로 구분된 값을 가집니다.
각 지표의 세부 분류 기준은 별도의 문서에서 제공될 예정입니다.

- Q1: 기상 직후 본인이 인지한 전반적인 수면의 질
 - 0: 개인 평균 이하
 - 1: 개인 평균 이상
- Q2: 취침 직전 본인이 느낀 신체적 피로 수준
 - 0: 높은 피로 수준
 - 1: 낮은 피로 수준
- Q3: 취침 직전 본인이 느낀 스트레스 수준
 - 0: 높은 스트레스 수준
 - 1: 낮은 스트레스 수준
- S1: 총 수면 시간(TST) 가이드라인을 준수했는지 3LEVELS
 - 0: 가이드라인 미준수
 - 1: 가이드라인 부분적 준수
 - 2: 가이드라인 완전 준수
- S2: 수면 효율(SE) 가이드라인을 준수했는지 여부
- (SE: 잠자리에 누워 있었던 전체 시간 대비, 실제로 잠든 시간의 비율)
 - 0: 가이드라인 미준수
 - 1: 가이드라인 준수
- S3: 수면 잠들기 지연 시간(SOL 또는 SL) 가이드라인을 준수했는지 여부
- (SOL: 잠자리에 누운 순간부터 실제로 잠드는 데까지 걸린 시간)
 - 0: 가이드라인 미준수
 - 1: 가이드라인 준수

### 📦 라이브러리

In [ ]:
! pip install haversine
! pip install optuna
! pip install category_encoders
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
from tqdm.auto import tqdm
from collections import Counter
from scipy.stats import entropy
from haversine import haversine  # 설치 필요: pip install haversine

warnings.filterwarnings('ignore')

In [ ]:
import re
import ast
from tqdm import tqdm  # ← 추가
from math import radians, cos, sin, asin, sqrt
from datetime import time
from datetime import timedelta
from functools import reduce
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import glob
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
import lightgbm as lgb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier, log_evaluation, early_stopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# seed 고정
SD = 42
random.seed(SD)
np.random.seed(SD)
os.environ['PYTHONHASHSEED'] = str(SD)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# pandas 옵션
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%0.4f' % x)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(1)

In [ ]:
def calculate_circular_mean_sleep_time(sleep_times):
    sleep_times = pd.Series(sleep_times).dropna()
    if len(sleep_times) == 0:
        return np.nan  # 혹은 return 0.0 등 기본값 설정 가능

    def hour_to_radian(hour):
        return (hour % 24) / 24 * 2 * np.pi

    radians = np.array([hour_to_radian(t) for t in sleep_times])
    mean_radian = np.arctan2(np.mean(np.sin(radians)), np.mean(np.cos(radians)))
    mean_hour = (mean_radian / (2 * np.pi)) * 24 % 24

    return mean_hour

In [ ]:
def circular_mean_sleep_time(times):

    # 결측치 제거
    valid_times = [t for t in times if pd.notna(t)]

    # 유효 데이터 개수 확인
    if len(valid_times) == 0:
        return None  # 결측치만 있는 경우

    # 시간 → 라디안 변환
    radians = [(t % 24) / 24 * 2 * np.pi for t in valid_times]

    # 사인/코사인 평균 계산
    sin_sum = np.mean(np.sin(radians))
    cos_sum = np.mean(np.cos(radians))

    # 평균 각도 계산
    if sin_sum == 0 and cos_sum == 0:
        return np.nan  # 불가능한 경우

    mean_radian = np.arctan2(sin_sum, cos_sum)

    # 평균 시간으로 변환
    mean_hour = (mean_radian / (2 * np.pi)) * 24
    if mean_hour < 0:
        mean_hour += 24

    return f'{int(mean_hour):02d}:{int((mean_hour % 1) * 60):02d}'

In [ ]:
def calculate_sleep_duration_min(sleep_time, wake_time):
    """
    취침 시각(sleep_time)과 기상 시각(wake_time)을 입력받아 수면 시간(분) 반환
    단위는 float 시간 (예: 23.5, 6.25)
    """
    if pd.isna(sleep_time) or pd.isna(wake_time):
        return None
    if wake_time < sleep_time:
        wake_time += 24  # 자정 넘긴 경우 보정
    duration = (wake_time - sleep_time) * 60
    return round(duration)

In [ ]:
def fill_missing_dates_by_subject(df, date_col='lifelog_date'):

    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    result = []

    for sid, group in df.groupby('subject_id'):
        group = group.sort_values(date_col)

        # 연속 날짜 생성
        full_dates = pd.date_range(start=group[date_col].min(), end=group[date_col].max())
        full_df = pd.DataFrame({date_col: full_dates})
        full_df['subject_id'] = sid

        # 병합
        merged = pd.merge(full_df, group, on=['subject_id', date_col], how='left')

        result.append(merged)

    # 병합 및 정렬
    final_df = pd.concat(result, ignore_index=True).sort_values(['subject_id', date_col])

    return final_df

In [ ]:
def get_time_block(hour):
    if 1 <= hour < 5:
        return 'sleeptime'
    else:
        return 'activehour'

In [ ]:
from io import StringIO

string = """
subject_id	sleep_date
id01	2024-07-24
id01	2024-07-27
id01	2024-08-18
id01	2024-08-19
id01	2024-08-20
id01	2024-08-21
id01	2024-08-22
id01	2024-08-24
id01	2024-08-25
id01	2024-08-26
id01	2024-08-27
id01	2024-08-28
id01	2024-08-29
id01	2024-08-30
id02	2024-08-23
id02	2024-08-24
id02	2024-09-16
id02	2024-09-17
id02	2024-09-19
id02	2024-09-20
id02	2024-09-21
id02	2024-09-22
id02	2024-09-23
id02	2024-09-24
id02	2024-09-25
id02	2024-09-26
id02	2024-09-27
id02	2024-09-28
id03	2024-08-30
id03	2024-09-01
id03	2024-09-02
id03	2024-09-03
id03	2024-09-05
id03	2024-09-06
id03	2024-09-07
id04	2024-09-03
id04	2024-09-04
id04	2024-09-05
id04	2024-09-06
id04	2024-09-07
id04	2024-09-08
id04	2024-09-09
id04	2024-10-08
id04	2024-10-09
id04	2024-10-10
id04	2024-10-11
id04	2024-10-12
id04	2024-10-13
id04	2024-10-14
id05	2024-10-19
id05	2024-10-23
id05	2024-10-24
id05	2024-10-25
id05	2024-10-26
id05	2024-10-27
id05	2024-10-28
id06	2024-07-25
id06	2024-07-26
id06	2024-07-27
id06	2024-07-28
id06	2024-07-29
id06	2024-07-30
id06	2024-07-31
id07	2024-07-07
id07	2024-07-08
id07	2024-07-09
id07	2024-07-10
id07	2024-07-11
id07	2024-07-12
id07	2024-07-13
id07	2024-07-30
id07	2024-08-01
id07	2024-08-02
id07	2024-08-03
id07	2024-08-04
id07	2024-08-05
id07	2024-08-06
id08	2024-08-28
id08	2024-08-29
id08	2024-08-30
id08	2024-08-31
id08	2024-09-01
id08	2024-09-02
id08	2024-09-04
id09	2024-08-02
id09	2024-08-22
id09	2024-08-23
id09	2024-08-24
id09	2024-08-25
id09	2024-08-27
id09	2024-08-28
id09	2024-08-29
id09	2024-08-30
id09	2024-08-31
id09	2024-09-01
id09	2024-09-02
id09	2024-09-03
id09	2024-09-04
id10	2024-08-28
id10	2024-08-30
id10	2024-08-31
id10	2024-09-01
id10	2024-09-02
id10	2024-09-03
id10	2024-09-06
"""

# DataFrame 생성
valid_ids = pd.read_csv(StringIO(string), sep='\t')
valid_ids['pk'] = valid_ids['subject_id']+valid_ids['sleep_date']

### 📦 데이터 읽기

In [ ]:
path = '/content/drive/MyDrive/data/ch2025_data_items/'

# 1
mACStatus = pd.read_parquet(path+'ch2025_mACStatus.parquet')
mActivity = pd.read_parquet(path+'ch2025_mActivity.parquet')
mAmbience = pd.read_parquet(path+'ch2025_mAmbience.parquet')
mBle = pd.read_parquet(path+'ch2025_mBle.parquet')
mGps = pd.read_parquet(path+'ch2025_mGps.parquet')
mLight = pd.read_parquet(path+'ch2025_mLight.parquet')
mScreenStatus = pd.read_parquet(path+'ch2025_mScreenStatus.parquet')
mUsageStats = pd.read_parquet(path+'ch2025_mUsageStats.parquet')
mWifi = pd.read_parquet(path+'ch2025_mWifi.parquet')
wHr = pd.read_parquet(path+'ch2025_wHr.parquet')
wLight = pd.read_parquet(path+'ch2025_wLight.parquet')
wPedo = pd.read_parquet(path+'ch2025_wPedo.parquet')

# 2
train = pd.read_csv('/content/drive/MyDrive/data/ch2025_metrics_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/ch2025_submission_sample.csv')

### 데이터 증강

In [ ]:
# !git clone https://github.com/amazon-science/tabsyn.git /content/drive/MyDrive/tabsyn

# 모델 학습

### ✔️ mACStatus 핸드폰 충전상태
- Indicates whether the smartphone is currently being charged.
- m_charging : 0/1 상태
- 핸드폰이 오랫 동안 충전했다는 의미?
 - 한 자리에 장시간 머물러 있었다.
 - 핸드폰을 장시간 사용하지 않았다.  

In [ ]:
mACStatus['lifelog_date'] = mACStatus['timestamp'].astype(str).str[:10]
# mACStatus = fill_missing_dates_by_subject(mACStatus)
mACStatus.head(1)

,subject_id,timestamp,m_charging,lifelog_date
0,id01,2024-06-26 12:03:00,0,2024-06-26


In [ ]:
def process_mACStatus(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])

    results = []

    for (subj, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        status = group['m_charging'].values  # 0/1 상태
        times = group['timestamp'].values

        ratio_charging = status.mean()
        sum_charging = status.sum()

        # 상태 전이 횟수
        transitions = (status[1:] != status[:-1]).sum()

        # 연속된 1 상태 길이들
        lengths = []
        current_len = 0
        for val in status:
            if val == 1:
                current_len += 1
            elif current_len > 0:
                lengths.append(current_len)
                current_len = 0
        if current_len > 0:
            lengths.append(current_len)

        avg_charging_duration = np.mean(lengths) if lengths else 0
        max_charging_duration = np.max(lengths) if lengths else 0

        results.append({
            'subject_id': subj,
            'lifelog_date': lifelog_date,
            'charging_ratio': ratio_charging,
            'charging_sum': sum_charging,
            'charging_transitions': transitions,
            'avg_charging_duration': avg_charging_duration,
            'max_charging_duration': max_charging_duration,
        })

    return pd.DataFrame(results)

mACStatus2 = process_mACStatus(mACStatus)

# check
print(f'# mACStatus2 shape: {mACStatus2.shape}')
mACStatus2.head(1)

# mACStatus2 shape: (700, 7)


,subject_id,lifelog_date,charging_ratio,charging_sum,charging_transitions,avg_charging_duration,max_charging_duration
0,id01,2024-06-26,0.2159,147,22,13.3636,41


### ✔️ mActivity 추정행동
- Value calculated by the Google Activity Recognition API.
 - 0 : IN_VEHICLE
 - 1 : ON_BICYCLE
 - 2 : ON_FOOT
 - 3 : STILL (not moving)
 - 4 : UNKNOWN
 - 5 : TILTING (This often occurs when a device is picked up from a desk or a user who is sitting stands up.)
 - 7 : WALKING
 - 8 : RUNNING
- 근무시간   : 오전 7시부터 오후 6시까지
- 근무외시간 : 오후6시부터 12시까지

In [ ]:
mActivity['lifelog_date'] = mActivity['timestamp'].astype(str).str[:10]
# mActivity = fill_missing_dates_by_subject(mActivity)
mActivity.head()

,subject_id,timestamp,m_activity,lifelog_date
0,id01,2024-06-26 12:03:00,4,2024-06-26
1,id01,2024-06-26 12:04:00,0,2024-06-26
2,id01,2024-06-26 12:05:00,0,2024-06-26
3,id01,2024-06-26 12:06:00,0,2024-06-26
4,id01,2024-06-26 12:07:00,0,2024-06-26


In [ ]:
def process_mActivity(df):
    """
    # 포함
    - 0 : IN_VEHICLE
    - 1 : ON_BICYCLE
    - 2 : ON_FOOT
    - 5 : TILTING (This often occurs when a device is picked up from a desk or a user who is sitting stands up.)
    - 7 : WALKING
    - 8 : RUNNING

    # 제외
    - 3 : STILL (not moving)
    - 4 : UNKNOWN
    """
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['hour'] = df['timestamp'].dt.hour

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'lifelog_date']):
        row = {'subject_id': subj, 'lifelog_date': date}

        # 전체 시간에서 1,2,7,8
        a1 = group[group['m_activity'].isin([1,2,7,8])]
        row['all_WALKING_n_ETC_minutes'] = len(a1)

        # 전체 시간에서 0 (IN_VEHICLE)
        a2 = group[group['m_activity'].isin([0])]
        row['all_VEHICLE_minutes'] = len(a2)

        # 전체 시간에서 유효한 활동
        all_valid = group[group['m_activity'].isin([0, 1, 2, 5, 7, 8])]
        row['all_ACTIVITY_minutes'] = len(all_valid)

        # sleeptime 0~5시 에서 유효한 활동
        dawn_valid = all_valid[(all_valid['hour'] >= 0) & (all_valid['hour'] <5)]
        row['dawn_ACTIVITY_minutes'] = len(dawn_valid)

        results.append(row)

    return pd.DataFrame(results)

In [ ]:
mActivity21 = process_mActivity(mActivity)

# check
print(f'# mActivity21 shape: {mActivity21.shape}')
mActivity21.head(1)

# mActivity21 shape: (700, 6)


,subject_id,lifelog_date,all_WALKING_n_ETC_minutes,all_VEHICLE_minutes,all_ACTIVITY_minutes,dawn_ACTIVITY_minutes
0,id01,2024-06-26,32,89,121,0


### ✔️ mActivity 추정행동2

In [ ]:
mActivity = pd.read_parquet(path+'ch2025_mActivity.parquet')
mActivity['lifelog_date'] = mActivity['timestamp'].astype(str).str[:10]
# mActivity = fill_missing_dates_by_subject(mActivity)
# mActivity['m_activity'] = mActivity['m_activity'].fillna(-1).astype(int)

In [ ]:
# 활동 데이터 원-핫 인코딩
"""활동 코드(m_activity)를 원-핫 인코딩하여 각 활동 유형별 컬럼 생성"""

mActivity = pd.merge(
    mActivity,
    pd.get_dummies(mActivity, columns=["m_activity"], prefix="m_activity", dtype=int),
    how="left",
    on=["subject_id", "timestamp","lifelog_date"],
)

In [ ]:
# 데이터 집계 함수 정의
def fn_love_aespa(
    df_input: pd.DataFrame, # 입력 데이터프레임
    str_value_col: str, # 집계할 컬럼명
    str_agg_func: str = "mean", # 집계 함수 (mean, median, mode, min, max, std, sum)
    str_freq: str = "30min", # 시간 간격 (30min, 60min, 120min, 240min, 360min 등)
) -> pd.DataFrame:
    # 데이터프레임 복사 및 timestamp 열을 datetime 형식으로 변환
    df_input_copy = df_input.copy()
    df_input_copy["timestamp"] = pd.to_datetime(df_input_copy["timestamp"])

    # 집계 결과 컬럼명 생성: @컬럼명@시간간격@집계함수
    str_agg_col_name = f"@{str_value_col}@{str_freq}@{str_agg_func}"

    # 집계 함수 설정 (mode는 별도 처리 필요)
    dict_aggregation = {}
    if str_agg_func == "mode":
        mode_agg_func = lambda x: (x.mode().iloc[0] if not x.mode().empty else np.nan)
        dict_aggregation[str_agg_col_name] = (str_value_col, mode_agg_func)
    else:
        dict_aggregation[str_agg_col_name] = (str_value_col, str_agg_func)

    # 그룹별 데이터 집계 수행
    df_agg = (
        df_input_copy.groupby(["subject_id", pd.Grouper(key="timestamp", freq=str_freq)]).agg(**dict_aggregation).reset_index()
    )

    # 날짜 및 시간 정보 추출
    df_agg["lifelog_date"] = df_agg["timestamp"].dt.date.astype(str)
    df_agg["hh24mi"] = df_agg["timestamp"].dt.strftime("%Hh%Mm")

    # 피벗 테이블로 데이터 재구성 (subject_id, lifelog_date 기준으로 시간대별 값 배치)
    df_pivot = df_agg.pivot_table(
        index=["subject_id", "lifelog_date"],
        columns="hh24mi",
        values=str_agg_col_name,
    )

    # 컬럼 이름 재구성 및 인덱스 초기화
    list_hh23mi_col = list(df_pivot.columns)
    df_pivot = df_pivot.reindex(columns=list_hh23mi_col).reset_index()
    list_hour_col = {hh24mi: f"{str_value_col}@{str_freq}@{str_agg_func}@{hh24mi}" for hh24mi in list_hh23mi_col}
    df_pivot = df_pivot.rename(columns=list_hour_col)

    return df_pivot

In [ ]:
# MET 값 매핑
"""
각 활동 코드에 해당하는 MET(Metabolic Equivalent of Task) 값 할당
MET는 신체 활동의 에너지 소비량을 측정하는 단위

활동 코드별 MET 값:
    0: 1.3 MET (가벼운 좌식 활동)
    1: 8.0 MET (격렬한 활동)
    3: 1.2 MET (매우 가벼운 활동)
    4: 3.0 MET (중간 강도 활동)
    7: 3.5 MET (중간 강도 활동)
    8: 10.0 MET (매우 격렬한 활동)
"""

dict_met_value = {0: 1.3, 1: 8.0, 3: 1.2, 4: 3.0, 7: 3.5, 8: 10.0}
for activity, met in dict_met_value.items():
    mActivity.loc[mActivity["m_activity"].isin([activity]), "m_activity_met"] = met

mActivity.head(5)

,subject_id,timestamp,m_activity,lifelog_date,m_activity_0,m_activity_1,m_activity_3,m_activity_4,m_activity_7,m_activity_8,m_activity_met
0,id01,2024-06-26 12:03:00,4,2024-06-26,0,0,0,1,0,0,3.0000
1,id01,2024-06-26 12:04:00,0,2024-06-26,1,0,0,0,0,0,1.3000
2,id01,2024-06-26 12:05:00,0,2024-06-26,1,0,0,0,0,0,1.3000
3,id01,2024-06-26 12:06:00,0,2024-06-26,1,0,0,0,0,0,1.3000
4,id01,2024-06-26 12:07:00,0,2024-06-26,1,0,0,0,0,0,1.3000


In [ ]:
# 활동 데이터 집계 - 활동 코드 모드(최빈값)
df_agg_activity = fn_love_aespa(df_input=mActivity,
                                str_value_col="m_activity",
                                # "mean", "median", "mode", "min", "max", "std"
                                str_agg_func="std",
                                # "30min", "60min", "120min", "240min", "360min", "480min", "720min", "1440min"
                                str_freq="240min",
                                )

df_agg_activity.head(5)

hh24mi,subject_id,lifelog_date,m_activity@240min@std@00h00m,m_activity@240min@std@04h00m,m_activity@240min@std@08h00m,m_activity@240min@std@12h00m,m_activity@240min@std@16h00m,m_activity@240min@std@20h00m
0,id01,2024-06-26,NaN,NaN,NaN,1.2175,1.6127,1.2744
1,id01,2024-06-27,0.0000,1.4930,1.0152,0.4869,1.5457,1.6453
2,id01,2024-06-28,0.0000,1.1278,1.3731,0.9862,1.6843,0.6007
3,id01,2024-06-29,0.0000,1.2239,0.6563,1.5931,0.7712,0.0000
4,id01,2024-06-30,0.0000,0.0000,1.4995,1.5716,0.7277,1.2385


In [ ]:
# 활동 데이터 집계 - MET 평균값
df_agg_activity_met = fn_love_aespa(df_input=mActivity,
                                    str_value_col="m_activity_met",
                                    # "mean", "median", "mode", "sum", "min", "max", "std"
                                    str_agg_func="std",
                                    # "30min", "60min", "120min", "240min", "360min", "480min", "720min", "1440min"
                                    str_freq="240min",
                                    )

df_agg_activity_met.head(5)

hh24mi,subject_id,lifelog_date,m_activity_met@240min@std@00h00m,m_activity_met@240min@std@04h00m,m_activity_met@240min@std@08h00m,m_activity_met@240min@std@12h00m,m_activity_met@240min@std@16h00m,m_activity_met@240min@std@20h00m
0,id01,2024-06-26,NaN,NaN,NaN,0.6120,0.5544,0.9151
1,id01,2024-06-27,0.0000,0.8631,0.4816,0.2561,0.3858,0.3269
2,id01,2024-06-28,0.0000,0.2953,0.6438,0.5458,0.3843,0.0200
3,id01,2024-06-29,0.0000,0.3290,0.2100,0.6038,0.0257,0.0000
4,id01,2024-06-30,0.0000,0.0000,0.3282,0.3856,0.0243,0.0413


In [ ]:
# 활동 데이터 집계 - m_activity_0, m_activity_1, m_activity_3, m_activity_4, m_activity_7, m_activity_8
df_agg_activity_0 = fn_love_aespa(df_input=mActivity,
                                  str_value_col="m_activity_0",
                                  # "mean", "median", "mode", "sum", "min", "max", "std"
                                  str_agg_func="std",
                                  # "30min", "60min", "120min", "240min", "360min", "480min", "720min", "1440min"
                                  str_freq="240min",
                                  )

df_agg_activity_0.head(5)

hh24mi,subject_id,lifelog_date,m_activity_0@240min@std@00h00m,m_activity_0@240min@std@04h00m,m_activity_0@240min@std@08h00m,m_activity_0@240min@std@12h00m,m_activity_0@240min@std@16h00m,m_activity_0@240min@std@20h00m
0,id01,2024-06-26,NaN,NaN,NaN,0.3078,0.3911,0.2737
1,id01,2024-06-27,0.0000,0.3808,0.1799,0.0645,0.4409,0.4918
2,id01,2024-06-28,0.0000,0.3266,0.2426,0.2571,0.4851,0.2002
3,id01,2024-06-29,0.0000,0.3494,0.1799,0.3578,0.2571,0.0000
4,id01,2024-06-30,0.0000,0.0000,0.4453,0.4495,0.2426,0.4128


In [ ]:
# 병합 기준 key
merge_keys = ['subject_id', 'lifelog_date']

# 세 개 데이터프레임 순차 병합
mActivity22 = (
    df_agg_activity
    .merge(df_agg_activity_met, on=merge_keys, how='outer')
    .merge(df_agg_activity_0, on=merge_keys, how='outer')
)

# check
print(mActivity22.shape)

(700, 20)


### ✔️ mAmbience 주변소리 (수정)
- Ambient sound identification labels and their respective probabilities.
- 무슨 소리가 난게 중요할까?
- 새벽에 무슨 소리던지 소리가 난게 중요한 걸까?
- 여러 가지 소리 중에 노이즈도 포함되어 있을까?

In [ ]:
def extract_labels_and_probs(row):
    items = row['m_ambience']
    labels = [item[0] for item in items]
    probs = [item[1] for item in items]
    return pd.Series({'labels': labels, 'prob': probs})

mAmbience = pd.read_parquet(path+'ch2025_mAmbience.parquet')
mAmbience[['labels', 'prob']]  = mAmbience.apply(extract_labels_and_probs, axis=1)
mAmbience['lifelog_date'] = mAmbience['timestamp'].astype(str).str[:10]
# mAmbience = fill_missing_dates_by_subject(mAmbience)
# mAmbience = mAmbience.drop(columns=['m_ambience'])
mAmbience.head(1)

,subject_id,timestamp,m_ambience,labels,prob,lifelog_date
0,id01,2024-06-26 13:00:10,"[[Music, 0.30902618], [Vehicle, 0.081680894], [Motor vehicle (road), 0.04035286], [Outside, urban or manmade, 0.037144363], [Outside, rural or natural, 0.032663062], [Car, 0.03199804], [Speech, 0.029806137], [Inside, large room or hall, 0.01684492], [Truck, 0.016206821], [Sound effect, 0.01591479]]","[Music, Vehicle, Motor vehicle (road), Outside, urban or manmade, Outside, rural or natural, Car, Speech, Inside, large room or hall, Truck, Sound effect]","[0.30902618, 0.081680894, 0.04035286, 0.037144363, 0.032663062, 0.03199804, 0.029806137, 0.01684492, 0.016206821, 0.01591479]",2024-06-26


In [ ]:
def process_mAmbience(df):

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])  # <- 이 줄 추가!
    df['hour'] = df['timestamp'].dt.hour

    # 시간대 분류
    df['time_period'] = df['timestamp'].dt.hour.map(get_time_block)

    # sleeptime은 lifelog_date 하루 차감
    df.loc[df['time_period'] == 'sleeptime', 'lifelog_date'] -= pd.Timedelta(days=1)

    # explode labels
    exploded = df.explode('labels')

    # unique label count
    unique_labels = (
        exploded.groupby(['subject_id', 'lifelog_date', 'time_period'])['labels']
        .nunique()
        .reset_index(name='unique_label_count')
    )

    # snor 포함 라벨 count
    snor_labels = (
        exploded[exploded['labels'].astype(str).str.contains('snor', case=False, na=False)]
        .groupby(['subject_id', 'lifelog_date', 'time_period'])['labels']
        .count()
        .reset_index(name='snor_count')
    )

    # 병합
    result = pd.merge(unique_labels, snor_labels, on=['subject_id', 'lifelog_date', 'time_period'], how='outer').fillna(0)
    result['snor_count'] = result['snor_count'].astype(int)
    result = result.pivot(index=['subject_id', 'lifelog_date'], columns='time_period')
    result.columns = [f"{tp}_{metric}" for metric, tp in result.columns]
    result = result.reset_index()

    return result

In [ ]:
mAmbience2 = process_mAmbience(mAmbience)

# check
print(f'# mAmbience2 shape: {mAmbience2.shape}')
mAmbience2.head(1)

# mAmbience2 shape: (801, 6)


,subject_id,lifelog_date,activehour_unique_label_count,sleeptime_unique_label_count,activehour_snor_count,sleeptime_snor_count
0,id01,2024-06-26,265.0000,10.0000,4.0000,0.0000


### ✔️ mBle 블루투스 (수정)
- Bluetooth devices around individual subject.
 - 7936 : Wearable, Headset, AV Device
 - 1796 : Peripheral (입력장치) 계열
 - 0 : 정보 없음 또는 알 수 없음(Unknown)
 - 1084 : Audio/Video (스피커, 헤드셋, 이어폰, TV 등)
 - 524 : Phone (휴대폰, 스마트폰)
 - 1060 : Headphones
 - 284 : commputer (PC, 노트북, PDA)

In [ ]:
def extract_mble_info(row):
    m_data = row['m_ble']
    address = [item['address'] for item in m_data]
    device_class = [item['device_class'] for item in m_data]
    rssi = [item['rssi'] for item in m_data]
    return pd.Series({'address': address, 'device_class': device_class, 'rssi': rssi})

mBle = pd.read_parquet(path+'ch2025_mBle.parquet')
mBle[['address','device_class','rssi']] = mBle.apply(extract_mble_info, axis=1)
mBle['lifelog_date'] = mBle['timestamp'].astype(str).str[:10]
# mBle = fill_missing_dates_by_subject(mBle)
# mBle['device_class'] = mBle['device_class'].fillna(0)
# mBle['rssi'] = mBle['rssi'].fillna(-1)
mBle.head(1)

,subject_id,timestamp,m_ble,address,device_class,rssi,lifelog_date
0,id01,2024-06-26 12:13:00,"[{'address': '00:15:7C:11:80:8D', 'device_class': '0', 'rssi': -82}, {'address': '01:B1:D2:20:9E:3A', 'device_class': '0', 'rssi': -61}, {'address': '04:33:1F:D9:C1:50', 'device_class': '0', 'rssi': -86}, {'address': '06:5C:2D:BC:39:BE', 'device_class': '0', 'rssi': -75}, {'address': '09:42:21:0D:AD:DF', 'device_class': '0', 'rssi': -70}, {'address': '0B:66:0D:D5:9C:4A', 'device_class': '0', 'rssi': -89}, {'address': '10:B5:88:E7:85:69', 'device_class': '0', 'rssi': -89}, {'address': '13:F0:CA:3B:DB:EF', 'device_class': '0', 'rssi': -77}, {'address': '1A:23:C0:8F:43:4D', 'device_class': '0', 'rssi': -66}, {'address': '24:11:53:BB:62:89', 'device_class': '1796', 'rssi': -37}, {'address': '24:2D:F0:EE:1E:D0', 'device_class': '0', 'rssi': -85}, {'address': '26:0C:48:28:15:77', 'device_class': '0', 'rssi': -63}, {'address': '27:C1:C0:8B:82:C9', 'device_class': '0', 'rssi': -88}, {'address': '28:9C:11:73:39:05', 'device_class': '0', 'rssi': -30}, {'address': '34:40:DE:35:F8:65', 'device_class': '0', 'rssi': -93}, {'address': '35:0A:59:BF:75:F5', 'device_class': '0', 'rssi': -72}, {'address': '41:A6:C4:20:E3:2C', 'device_class': '7936', 'rssi': -83}, {'address': '42:6B:51:95:1B:D4', 'device_class': '0', 'rssi': -77}, {'address': '44:B2:0B:78:04:0F', 'device_class': '0', 'rssi': -69}, {'address': '45:37:48:E2:7F:CC', 'device_class': '0', 'rssi': -87}, {'address': '4E:1B:C2:DF:C5:87', 'device_class': '0', 'rssi': -76}, {'address': '4E:9F:1B:A9:56:5D', 'device_class': '0', 'rssi': -66}, {'address': '50:63:B0:82:07:00', 'device_class': '0', 'rssi': -86}, {'address': '53:13:6C:4F:04:D2', 'device_class': '0', 'rssi': -69}, {'address': '54:15:89:95:27:44', 'device_class': '7936', 'rssi': -71}, {'address': '56:0E:2E:B0:D4:11', 'device_class': '0', 'rssi': -61}, {'address': '5A:7A:2E:42:03:B1', 'device_class': '0', 'rssi': -82}, {'address': '5A:9D:3E:AB:38:C6', 'device_class': '0', 'rssi': -83}, {'address': '5E:A6:8E:B8:74:74', 'device_class': '0', 'rssi': -84}, {'address': '5F:BC:08:0F:C1:6A', 'device_class': '0', 'rssi': -87}, {'address': '62:E1:9D:41:F4:AE', 'device_class': '0', 'rssi': -73}, {'address': '67:23:FE:88:69:A8', 'device_class': '0', 'rssi': -88}, {'address': '68:EC:C5:0C:D1:C1', 'device_class': '0', 'rssi': -78}, {'address': '6B:28:DA:C0:1B:29', 'device_class': '0', 'rssi': -75}, {'address': '6F:0B:91:00:33:19', 'device_class': '0', 'rssi': -80}, {'address': '70:7A:4B:82:44:90', 'device_class': '0', 'rssi': -88}, {'address': '7B:62:D4:5B:59:D3', 'device_class': '0', 'rssi': -74}, {'address': '7B:BE:A4:9D:FD:11', 'device_class': '0', 'rssi': -72}, {'address': '7F:FD:C4:00:77:7D', 'device_class': '0', 'rssi': -52}, {'address': 'C4:F0:92:C8:F1:8D', 'device_class': '7936', 'rssi': -87}, {'address': 'C7:3F:2C:7B:86:66', 'device_class': '7936', 'rssi': -89}]","[00:15:7C:11:80:8D, 01:B1:D2:20:9E:3A, 04:33:1F:D9:C1:50, 06:5C:2D:BC:39:BE, 09:42:21:0D:AD:DF, 0B:66:0D:D5:9C:4A, 10:B5:88:E7:85:69, 13:F0:CA:3B:DB:EF, 1A:23:C0:8F:43:4D, 24:11:53:BB:62:89, 24:2D:F0:EE:1E:D0, 26:0C:48:28:15:77, 27:C1:C0:8B:82:C9, 28:9C:11:73:39:05, 34:40:DE:35:F8:65, 35:0A:59:BF:75:F5, 41:A6:C4:20:E3:2C, 42:6B:51:95:1B:D4, 44:B2:0B:78:04:0F, 45:37:48:E2:7F:CC, 4E:1B:C2:DF:C5:87, 4E:9F:1B:A9:56:5D, 50:63:B0:82:07:00, 53:13:6C:4F:04:D2, 54:15:89:95:27:44, 56:0E:2E:B0:D4:11, 5A:7A:2E:42:03:B1, 5A:9D:3E:AB:38:C6, 5E:A6:8E:B8:74:74, 5F:BC:08:0F:C1:6A, 62:E1:9D:41:F4:AE, 67:23:FE:88:69:A8, 68:EC:C5:0C:D1:C1, 6B:28:DA:C0:1B:29, 6F:0B:91:00:33:19, 70:7A:4B:82:44:90, 7B:62:D4:5B:59:D3, 7B:BE:A4:9D:FD:11, 7F:FD:C4:00:77:7D, C4:F0:92:C8:F1:8D, C7:3F:2C:7B:86:66]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1796, 0, 0, 0, 0, 0, 0, 7936, 0, 0, 0, 0, 0, 0, 0, 7936, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7936, 7936]","[-82, -61, -86, -75, -70, -89, -89, -77, -66, -37, -85, -63, -88, -30, -93, -72, -83, -77, -69, -87, -76, -66, -86, -69, -71, -61, -82, -83, -84, -87, -73, -88, -78, -75, -80, 

In [ ]:
def process_mBle(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    df['time_period'] = df['timestamp'].dt.hour.map(get_time_block)

    features = []

    for idx, row in df.iterrows():
        entry = ast.literal_eval(row['m_ble']) if isinstance(row['m_ble'], str) else row['m_ble']

        rssi_list = []
        class_0_cnt = 0
        class_other_cnt = 0

        for device in entry:
            try:
                rssi = int(device['rssi'])
                rssi_list.append(rssi)

                device_class = str(device['device_class'])
                if device_class == '0':
                    class_0_cnt += 1
                else:
                    class_other_cnt += 1
            except:
                continue  # malformed record

        feature = {
            'subject_id': row['subject_id'],
            'lifelog_date': row['lifelog_date'],
            'time_period': row['time_period'],
            'ble_class_unknwn_cnt': class_0_cnt,
            'ble_class_others_cnt': class_other_cnt,
            'ble_count': len(rssi_list),
            'ble_rssi_mean': np.mean(rssi_list) if rssi_list else np.nan,
            'ble_rssi_min': np.min(rssi_list) if rssi_list else np.nan,
            'ble_rssi_max': np.max(rssi_list) if rssi_list else np.nan,
        }
        features.append(feature)

    return pd.DataFrame(features)

def summarize_mBle_daily(df):

    # row 단위 BLE feature 추출
    df = process_mBle(df)

    # sleeptime은 lifelog_date 하루 차감
    df.loc[df['time_period'] == 'sleeptime', 'lifelog_date'] -= pd.Timedelta(days=1)

    # 하루 + 시간대별로 groupby
    grouped = df.groupby(['subject_id', 'lifelog_date', 'time_period']).agg({
        'ble_class_unknwn_cnt': 'sum',
        'ble_class_others_cnt': 'sum',
        'ble_rssi_mean': 'mean',
        'ble_rssi_min': 'min',
        'ble_rssi_max': 'max',
    }).reset_index()

    # 총합 구해서 비율 계산
    total_cnt = grouped['ble_class_unknwn_cnt'] + grouped['ble_class_others_cnt']
    grouped['ble_class_unknwn_ratio'] = grouped['ble_class_unknwn_cnt'] / total_cnt.replace(0, np.nan)
    grouped['ble_class_others_ratio'] = grouped['ble_class_others_cnt'] / total_cnt.replace(0, np.nan)

    # 필요 없는 cnt 컬럼 제거
    grouped.drop(columns=[
        'ble_class_unknwn_cnt',
        'ble_class_others_cnt'
    ], inplace=True)

    # pivot해서 time_period별로 펼치기
    final = grouped.pivot(index=['subject_id', 'lifelog_date'], columns='time_period')
    final.columns = ['_'.join(col).strip() for col in final.columns.values]
    final = final.reset_index()

    return final

In [ ]:
mBle2 = summarize_mBle_daily(mBle)

# check
print(f'\n # mBle2 shape: {mBle2.shape}')
mBle2.head(1)


 # mBle2 shape: (696, 12)


,subject_id,lifelog_date,ble_rssi_mean_activehour,ble_rssi_mean_sleeptime,ble_rssi_min_activehour,ble_rssi_min_sleeptime,ble_rssi_max_activehour,ble_rssi_max_sleeptime,ble_class_unknwn_ratio_activehour,ble_class_unknwn_ratio_sleeptime,ble_class_others_ratio_activehour,ble_class_others_ratio_sleeptime
0,id01,2024-06-26,-75.6681,NaN,-94.0000,NaN,-27.0000,NaN,0.9313,NaN,0.0687,NaN


### ✔️ mGps, GPS 기반 핸드폰 위치
- Multiple GPS coordinates measured within a single minute using the smartphone.
- speed가 1보다 큰경우 정지 상태가 아니고 움직이고 있다고 판단
 - 0.5-2 : 걸어서 이동하는 경우  
 - 2-5 : 조깅
 - 5 이상 : 차를 타고 이동하는 경우

- speed가 0.5-2사이를 하루에 몇분동안 지속했는지?
- speed가 2-5사이를 하루에 몇분동안 지속했는지? (유산소 운동 시간)
- speed가 5이상을 하루에 몇분동안 지속했는지?  

In [ ]:
def extract_gps_info(row):
    m_data = row['m_gps']
    altitude = [item['altitude'] for item in m_data]
    latitude = [item['latitude'] for item in m_data]
    longitude = [item['longitude'] for item in m_data]
    speed = [item['speed'] for item in m_data]
    return pd.Series({'altitude': altitude, 'latitude': latitude, 'longitude': longitude, 'speed': speed})

mGps[['altitude','latitude','longitude','speed']] = mGps.apply(extract_gps_info, axis=1)
mGps['lifelog_date'] = mGps['timestamp'].astype(str).str[:10]
# mGps = fill_missing_dates_by_subject(mGps)
# mGps = mGps.drop(columns=['m_gps'])
mGps.head(1)

,subject_id,timestamp,m_gps,altitude,latitude,longitude,speed,lifelog_date
0,id01,2024-06-26 12:03:00,"[{'altitude': 110.6, 'latitude': 0.2077385, 'longitude': 0.170027, 'speed': 0.0}, {'altitude': 110.8, 'latitude': 0.2077759, 'longitude': 0.1699851, 'speed': 0.721}, {'altitude': 110.8, 'latitude': 0.2077728, 'longitude': 0.1699834, 'speed': 0.0505}, {'altitude': 110.7, 'latitude': 0.20779, 'longitude': 0.1699686, 'speed': 0.6587}, {'altitude': 110.7, 'latitude': 0.2077914, 'longitude': 0.1699708, 'speed': 0.0568}, {'altitude': 110.8, 'latitude': 0.2077972, 'longitude': 0.1699657, 'speed': 0.1768}, {'altitude': 110.8, 'latitude': 0.2078002, 'longitude': 0.1699627, 'speed': 0.0907}, {'altitude': 110.8, 'latitude': 0.2077985, 'longitude': 0.1699631, 'speed': 0.0337}, {'altitude': 110.8, 'latitude': 0.207801, 'longitude': 0.1699642, 'speed': 0.0411}, {'altitude': 110.8, 'latitude': 0.207802, 'longitude': 0.1699639, 'speed': 0.0296}, {'altitude': 110.8, 'latitude': 0.2078011, 'longitude': 0.169963, 'speed': 0.0194}]","[110.6, 110.8, 110.8, 110.7, 110.7, 110.8, 110.8, 110.8, 110.8, 110.8, 110.8]","[0.2077385, 0.2077759, 0.2077728, 0.20779, 0.2077914, 0.2077972, 0.2078002, 0.2077985, 0.207801, 0.207802, 0.2078011]","[0.170027, 0.1699851, 0.1699834, 0.1699686, 0.1699708, 0.1699657, 0.1699627, 0.1699631, 0.1699642, 0.1699639, 0.169963]","[0.0, 0.721, 0.0505, 0.6587, 0.0568, 0.1768, 0.0907, 0.0337, 0.0411, 0.0296, 0.0194]",2024-06-26


In [ ]:
# 거리 계산 함수
def haversine(coord1, coord2, unit='m'):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371000  # 지구 반지름(m)
    return c * r if unit == 'm' else c * r / 1000

def process_mGps(df):
    df = df.copy()

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['week'] = df['timestamp'].dt.isocalendar().week

    expanded_rows = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing GPS data"):
        speeds = ast.literal_eval(row['speed']) if isinstance(row['speed'], str) else row['speed']
        lats = ast.literal_eval(row['latitude']) if isinstance(row['latitude'], str) else row['latitude']
        lons = ast.literal_eval(row['longitude']) if isinstance(row['longitude'], str) else row['longitude']
        alts = ast.literal_eval(row['altitude']) if isinstance(row['altitude'], str) else row['altitude']
        n = len(speeds)
        if n > 0:
            expanded_rows.append(pd.DataFrame({
                'subject_id': [row['subject_id']] * n,
                'lifelog_date': [row['lifelog_date']] * n,
                'timestamp': pd.date_range(start=row['timestamp'], periods=n, freq='1S'),
                'speed': speeds,
                'latitude': lats,
                'longitude': lons,
                'altitude': alts
            }))

    expanded_df = pd.concat(expanded_rows, ignore_index=True)

    # 벡터화
    speeds = expanded_df['speed'].values

    walk_mask = (0.5 <= speeds) & (speeds < 2)
    jog_mask = (2 <= speeds) & (speeds < 5)
    vehicle_mask = (speeds >= 5)
    le5_mask = (speeds <= 5)

    expanded_df['walk'] = walk_mask.astype(int)
    expanded_df['jog'] = jog_mask.astype(int)
    expanded_df['vehicle'] = vehicle_mask.astype(int)
    expanded_df['le5_speed'] = expanded_df['speed'].where(le5_mask)

    # 아침/저녁 구간 조건
    expanded_df['hour'] = expanded_df['timestamp'].dt.hour
    morning_condition = (expanded_df['hour'] >= 6) & (expanded_df['hour'] < 9) & (expanded_df['speed'] >= 1)
    evening_condition = (expanded_df['hour'] >= 21) & (expanded_df['hour'] <= 23) & (expanded_df['speed'] <= 1)

    # 이동 특성 계산
    movement_features = []
    for (subject_id, lifelog_date), group in expanded_df.groupby(['subject_id', 'lifelog_date']):
        all_speeds = group['speed'].values
        all_alts = group['altitude'].values
        all_lats = group['latitude'].values
        all_lons = group['longitude'].values

        active_mins = group.shape[0] / 60  # 1초 단위 → 분
        movement_ratio = (all_speeds > 1.0).mean() if len(all_speeds) > 0 else 0
        alt_change = all_alts[-1] - all_alts[0] if len(all_alts) > 0 else 0
        lat_change = all_lats[-1] - all_lats[0] if len(all_lats) > 0 else 0
        lon_change = all_lons[-1] - all_lons[0] if len(all_lons) > 0 else 0

        total_dist = 0.0
        if len(all_lats) > 1:
            for i in range(len(all_lats)-1):
                coord1 = (all_lats[i], all_lons[i])
                coord2 = (all_lats[i+1], all_lons[i+1])
                total_dist += haversine(coord1, coord2, unit='m')

        movement_features.append({
            'subject_id': subject_id,
            'lifelog_date': lifelog_date,
            'active_minutes': active_mins,
            'movement_ratio': movement_ratio,
            'alt_change': alt_change,
            'lat_change': lat_change,
            'lon_change': lon_change,
            'total_distance_m': total_dist
        })

    movement_df = pd.DataFrame(movement_features)

    # Groupby + Aggregation
    agg_funcs = {
        'walk_minutes': ('walk', lambda x: x.sum() / 60),
        'jog_minutes': ('jog', lambda x: x.sum() / 60),
        'vehicle_minutes': ('vehicle', lambda x: x.sum() / 60),
        'speed_le5_max': ('le5_speed', 'max'),
        'speed_le5_mean': ('le5_speed', 'mean'),
        'speed_le5_std': ('le5_speed', 'std')
    }

    grouped = expanded_df.groupby(['subject_id', 'lifelog_date']).agg(**agg_funcs).reset_index()
    grouped['exercise_flag'] = np.where(grouped['jog_minutes'] >= 5,1,0)

    # 아침 wakeup time
    morning_first_movement = (
        expanded_df[morning_condition]
        .groupby(['subject_id', 'lifelog_date'])['timestamp']
        .min()
        .reset_index()
        .rename(columns={'timestamp': 'morning_wakeup_time'})
    )


    # 최종 merge
    final = pd.merge(grouped, movement_df, on=['subject_id', 'lifelog_date'], how='left')
    final = pd.merge(final, morning_first_movement, on=['subject_id', 'lifelog_date'], how='left')

    # 아침 wakeup_time 처리
    valid_wakeup = final['morning_wakeup_time'].dropna()
    if not valid_wakeup.empty:
        total_seconds = valid_wakeup.dt.hour * 3600 + valid_wakeup.dt.minute * 60 + valid_wakeup.dt.second
        mean_seconds = total_seconds.mean()
        mean_hour = int(mean_seconds // 3600)
        mean_minute = int((mean_seconds % 3600) // 60)
        mean_second = int(mean_seconds % 60)
        mean_wakeup_time = time(mean_hour, mean_minute, mean_second)
    else:
        mean_wakeup_time = time(7, 0, 0)

    final['morning_wakeup_time'] = final['morning_wakeup_time'].fillna(
        pd.Timestamp.combine(pd.to_datetime('today').date(), mean_wakeup_time)
    )
    final['morning_wakeup_time'] = final['morning_wakeup_time'].dt.hour * 100 + final['morning_wakeup_time'].dt.minute

    mean_wakeup_hhmm = mean_wakeup_time.hour * 100 + mean_wakeup_time.minute

    # wake_up_early_minutes
    def compute_minutes_diff(actual_hhmm, mean_hhmm):
        actual_hour = actual_hhmm // 100
        actual_minute = actual_hhmm % 100
        mean_hour = mean_hhmm // 100
        mean_minute = mean_hhmm % 100
        actual_sec = actual_hour * 3600 + actual_minute * 60
        mean_sec = mean_hour * 3600 + mean_minute * 60
        return (mean_sec - actual_sec) / 60

    final['wake_up_early_minutes'] = final['morning_wakeup_time'].apply(lambda x: compute_minutes_diff(x, mean_wakeup_hhmm))

    return final

In [ ]:
# %%time

# mGps2 = process_mGps(mGps)

# # check
# print(f'\n # mGps2 shape: {mGps2.shape}')
# mGps2.head(1)

### 🔥 mLight 주변 밝기
- Ambient light measured by the smartphone.
 - 어두운 밤	0.1 ~ 1 lux	캄캄한 방, 달빛 없는 밤
 - 가로등 켜진 거리	10 ~ 20 lux	흐릿한 외부 조명
 - 실내 조명	100 ~ 500 lux	사무실, 일반 거실
 - 밝은 실외	10,000 ~ 25,000 lux	맑은 날 햇빛
 - 직사광선 아래	30,000 ~ 100,000 lux	여름 한낮, 매우 강한 햇빛

- 밝기에 따라서 언제 불을 끄고 잠든 시간 추정
- 직사광선 잠에 좋은 영향을 주는지? (논문)
- 결측치 처리 x

In [ ]:
mLight['lifelog_date'] = mLight['timestamp'].astype(str).str[:10]
# mLight = fill_missing_dates_by_subject(mLight)
mLight.head()

,subject_id,timestamp,m_light,lifelog_date
0,id01,2024-06-26 12:03:00,534.0000,2024-06-26
1,id01,2024-06-26 12:13:00,846.0000,2024-06-26
2,id01,2024-06-26 12:23:00,826.0000,2024-06-26
3,id01,2024-06-26 12:33:00,851.0000,2024-06-26
4,id01,2024-06-26 12:43:00,428.0000,2024-06-26


In [ ]:
def process_mLight(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    df['is_night'] = df['hour'].apply(lambda h: h >= 22 or h < 6)

    # 하루 요약 통계
    daily_light = df.groupby(['subject_id', 'lifelog_date']).agg(
        light_mean=('m_light', 'mean'),
        light_std=('m_light', 'std'),
        light_max=('m_light', 'max'),
        light_min=('m_light', 'min'),
        light_night_mean=('m_light', lambda x: x[df.loc[x.index, 'is_night']].mean()),
        light_day_mean=('m_light', lambda x: x[~df.loc[x.index, 'is_night']].mean()),
        light_night_ratio=('is_night', 'mean')
    ).reset_index()

    results = []

    for subject_id, group in tqdm(df.groupby('subject_id'), desc="Processing light-based sleep detection"):
        group = group.sort_values('timestamp').reset_index(drop=True)

        recorded_dates = set()
        sleeping = False
        zero_count = 0
        first_zero_time = None

        for i in range(len(group)):
            light = group.loc[i, 'm_light']
            hour = group.loc[i, 'hour']

            if light == 0:
                zero_count += 1
                if zero_count == 1:
                    first_zero_time = group.loc[i, 'timestamp']
                if zero_count >= 6 and not sleeping:
                    sleep_hour = first_zero_time.hour
                    if (sleep_hour >= 21 or sleep_hour <= 2):
                        sleeping = True
            else:
                if sleeping:
                    candidate_wakeup = group.loc[i, 'timestamp']
                    wake_hour = candidate_wakeup.hour

                    if 5 <= wake_hour <= 9 and first_zero_time is not None:
                        wake_time = candidate_wakeup
                        sleep_time = first_zero_time
                        duration_min = (wake_time - sleep_time).total_seconds() / 60

                        if 0 < duration_min <= 840:
                            sleep_duration = duration_min
                        else:
                            sleep_duration = np.nan

                        lifelog_date = wake_time.date() + pd.Timedelta(days=-1)

                        if lifelog_date not in recorded_dates:
                            results.append({
                                'subject_id': subject_id,
                                'lifelog_date': lifelog_date,
                                'sleep_duration_min_mLight': sleep_duration,
                                'sleep_time_min_mLight': sleep_time.hour * 60 + sleep_time.minute,
                                'wake_time_min_mLight': wake_time.hour * 60 + wake_time.minute,
                                'hour_slept_mLight': sleep_time.hour + sleep_time.minute / 60,
                                'hour_woke_up_mLight': wake_time.hour + wake_time.minute / 60
                            })
                            recorded_dates.add(lifelog_date)

                        sleeping = False
                        zero_count = 0
                        first_zero_time = None

            if light > 0:
                zero_count = 0
                first_zero_time = None

    sleep_df = pd.DataFrame(results)

    # 정렬 + 보간
    sleep_df = sleep_df.sort_values(['subject_id', 'lifelog_date'])
    sleep_df['sleep_duration_interp_mLight'] = sleep_df.groupby('subject_id')['sleep_duration_min_mLight'].transform(lambda x: x.interpolate())

    # 시간 단위 파생 컬럼
    sleep_df['sleep_duration_hour_mLight'] = sleep_df['sleep_duration_min_mLight'] / 60
    sleep_df['sleep_duration_interp_hour_mLight'] = sleep_df['sleep_duration_interp_mLight'] / 60

    # 병합
    final = pd.merge(daily_light, sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    return final

In [ ]:
def process_mLight2(df):
    from datetime import datetime, timedelta

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    # m_light > 0 → m_screen_use로 변환
    df['m_light_on'] = (df['m_light'] > 0).astype(int)

    # base key 확보
    base_keys = df[['subject_id', 'lifelog_date']].drop_duplicates()
    base_keys['lifelog_date'] = base_keys['lifelog_date'].dt.date

    # 밤 9시 ~ 다음날 오전 11시 필터링
    df['hour'] = df['timestamp'].dt.hour
    df = df[(df['hour'] >= 21) | (df['hour'] < 11)].copy()
    df.loc[df['hour'] < 11, 'lifelog_date'] -= pd.Timedelta(days=1)

    df.sort_values(['subject_id', 'timestamp'], inplace=True)

    results = []

    for (subject_id, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        group = group.sort_values('timestamp').reset_index(drop=True)

        # 1. 중간 각성 제거
        prev = group['m_light_on'].shift(1)
        next_ = group['m_light_on'].shift(-1)
        mask = (group['m_light_on'] == 1) & (prev == 0) & (next_ == 0)
        group.loc[mask, 'm_light_on'] = 0

        # 2. 짧은 각성 블록 제거
        group['is_sleep'] = group['m_light_on'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        block_info = group.groupby('block').agg(
            is_sleep=('is_sleep', 'first'),
            size=('is_sleep', 'size')
        )

        for i in range(1, len(block_info) - 1):
            if (
                block_info.iloc[i]['is_sleep'] == False and
                block_info.iloc[i]['size'] <= 2 and
                block_info.iloc[i - 1]['is_sleep'] and
                block_info.iloc[i + 1]['is_sleep']
            ):
                group.loc[group['block'] == block_info.index[i], 'm_light_on'] = 0

        # 3. 수면 블록 추정
        group['is_sleep'] = group['m_light_on'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        sleep_blocks = group[group['is_sleep']].groupby('block').agg(
            sleep_start=('timestamp', 'first'),
            sleep_end=('timestamp', 'last'),
            duration_min=('timestamp', lambda x: (x.max() - x.min()).total_seconds() / 60)
        )

        sleep_time = wake_time = duration_min = None
        if not sleep_blocks.empty:
            longest_sleep = sleep_blocks.loc[sleep_blocks['duration_min'].idxmax()]
            sleep_time = longest_sleep['sleep_start'].time()
            wake_time = longest_sleep['sleep_end'].time()
            duration_min = longest_sleep['duration_min']

            # 유효성 조건
            if not (4 <= wake_time.hour < 11):
                wake_time = None
            if not (sleep_time.hour >= 21 or sleep_time.hour < 3):
                sleep_time = None
            if duration_min < 100:
                sleep_time = None
                wake_time = None
                duration_min = None

        results.append({
            'subject_id': subject_id,
            'lifelog_date': lifelog_date.date(),
            'sleep_time': sleep_time,
            'wake_time': wake_time,
            'sleep_duration_min': round(duration_min, 1) if duration_min is not None else None
        })

    sleep_df = pd.DataFrame(results)
    result_df = base_keys.merge(sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    # 시간 → 실수형 숫자 변환
    def time_to_float(t):
        if pd.isna(t):
            return None
        return round(t.hour + t.minute / 60 + t.second / 3600, 4)

    result_df['sleep_time'] = result_df['sleep_time'].apply(time_to_float)
    result_df['wake_time'] = result_df['wake_time'].apply(time_to_float)

    return result_df

In [ ]:
def add_ratios(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['weekday'] = df['lifelog_date'].dt.weekday
    df['week_type'] = df['weekday'].apply(lambda x: 'weekend' if x >= 5 else 'weekday')
    df['month'] = df['lifelog_date'].dt.month

    # 평균 계산
    avg_duration = df.groupby(['subject_id', 'month', 'week_type'])['sleep_duration_min'].mean().reset_index(name='avg_sleep_duration')
    sleep_time_avg = df.groupby(['subject_id', 'month', 'week_type'])['sleep_time'].apply(calculate_circular_mean_sleep_time).reset_index(name='avg_sleep_time')
    wake_time_avg = df.groupby(['subject_id', 'month', 'week_type'])['wake_time'].apply(calculate_circular_mean_sleep_time).reset_index(name='avg_wake_time')
    avg_df = sleep_time_avg.merge(wake_time_avg, on=['subject_id', 'month', 'week_type']).merge(avg_duration, on=['subject_id', 'month', 'week_type'])
    df = df.merge(avg_df, on=['subject_id', 'month', 'week_type'], how='left')

    # 비율 및 차이
    df['sleep_time_diff'] = df['avg_sleep_time'] - df['sleep_time']
    df['wake_time_diff'] = df['avg_wake_time'] - df['wake_time']
    df['sleep_duration_diff'] = df['avg_sleep_duration'] - df['sleep_duration_min']
    df['sleep_time_ratio'] = df['sleep_time'] / df['avg_sleep_time']
    df['wake_time_ratio'] = df['wake_time'] / df['avg_wake_time']
    df['sleep_duration_ratio'] = df['sleep_duration_min'] / df['avg_sleep_duration']

    # 정렬 후 lag/변화량
    df = df.sort_values(['subject_id', 'lifelog_date'])
    for lag in [1, 2]:
        df[f'sleep_time_lag{lag}'] = df.groupby('subject_id')['sleep_time'].shift(lag)
        df[f'wake_time_lag{lag}'] = df.groupby('subject_id')['wake_time'].shift(lag)
        df[f'sleep_duration_lag{lag}'] = df.groupby('subject_id')['sleep_duration_min'].shift(lag)
        df[f'sleep_time_diff_lag{lag}'] = df.groupby('subject_id')['sleep_time'].diff(lag)
        df[f'wake_time_diff_lag{lag}'] = df.groupby('subject_id')['wake_time'].diff(lag)
        df[f'sleep_duration_diff_lag{lag}'] = df.groupby('subject_id')['sleep_duration_min'].diff(lag)
    df['week_type_lag1'] = df.groupby('subject_id')['week_type'].shift(1)

    # 이동 평균 (2,3)
    for window in [2, 3]:
        df[f'rolling_sleep_time_{window}d'] = df.groupby('subject_id')['sleep_time'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)
        df[f'rolling_wake_time_{window}d'] = df.groupby('subject_id')['wake_time'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)
        df[f'rolling_sleep_duration_{window}d'] = df.groupby('subject_id')['sleep_duration_min'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)

    # 요일별 평균 수면 비교
    weekday_avg = df.groupby(['subject_id', 'weekday'])['sleep_duration_min'].mean().reset_index(name='weekday_avg_sleep')
    df = df.merge(weekday_avg, on=['subject_id', 'weekday'], how='left')
    df['sleep_duration_vs_weekday_avg'] = df['sleep_duration_min'] - df['weekday_avg_sleep']

    # 급격한 수면시간 변화 여부 (60분 이상 변화)
    df['is_sleep_duration_change_large'] = (df['sleep_duration_diff_lag1'].abs() > 60).astype(int)

    return df

In [ ]:
mLight21 = process_mLight(mLight)

# check
print(f'\n # mLight21 shape: {mLight21.shape}')
mLight21.head(1)

Processing light-based sleep detection: 100%|██████████| 10/10 [00:01<00:00,  6.25it/s]


 # mLight21 shape: (700, 17)


,subject_id,lifelog_date,light_mean,light_std,light_max,light_min,light_night_mean,light_day_mean,light_night_ratio,sleep_duration_min_mLight,sleep_time_min_mLight,wake_time_min_mLight,hour_slept_mLight,hour_woke_up_mLight,sleep_duration_interp_mLight,sleep_duration_hour_mLight,sleep_duration_interp_hour_mLight
0,id01,2024-06-26,364.5068,395.6594,1886.0000,0.0000,184.9231,403.4167,0.1781,340.0000,1409.0000,309.0000,23.4833,5.1500,340.0000,5.6667,5.6667


In [ ]:
mLight22 = process_mLight2(mLight)
mLight22['sleep_time'] = np.where(mLight22['sleep_time']<24,mLight22['sleep_time']+24,mLight22['sleep_time'])
mLight22['sleep_duration_min'] = mLight22.apply(lambda x: calculate_sleep_duration_min(x['sleep_time'],x['wake_time']),axis=1)
mLight22 = add_ratios(mLight22)
mLight22 = mLight22.drop(columns=['week_type','wake_time_lag1'])
mLight22.columns = ['subject_id', 'lifelog_date']+['light_'+i for i in mLight22.columns if i not in ['subject_id', 'lifelog_date']]
mLight22['lifelog_date'] = mLight22['lifelog_date'].astype(str)

# check
# mLight22 shape: (700, 55)
print(f'\n # mLight22 shape: {mLight22.shape}')
mLight22.head(1)


 # mLight22 shape: (700, 37)


,subject_id,lifelog_date,light_sleep_time,light_wake_time,light_sleep_duration_min,light_weekday,light_month,light_avg_sleep_time,light_avg_wake_time,light_avg_sleep_duration,light_sleep_time_diff,light_wake_time_diff,light_sleep_duration_diff,light_sleep_time_ratio,light_wake_time_ratio,light_sleep_duration_ratio,light_sleep_time_lag1,light_sleep_duration_lag1,light_sleep_time_diff_lag1,light_wake_time_diff_lag1,light_sleep_duration_diff_lag1,light_sleep_time_lag2,light_wake_time_lag2,light_sleep_duration_lag2,light_sleep_time_diff_lag2,light_wake_time_diff_lag2,light_sleep_duration_diff_lag2,light_week_type_lag1,light_rolling_sleep_time_2d,light_rolling_wake_time_2d,light_rolling_sleep_duration_2d,light_rolling_sleep_time_3d,light_rolling_wake_time_3d,light_rolling_sleep_duration_3d,light_weekday_avg_sleep,light_sleep_duration_vs_weekday_avg,light_is_sleep_duration_change_large
0,id01,2024-06-26,47.4833,4.9833,-1110.0000,2,6,23.3722,5.0944,-1096.6667,-24.1111,0.1111,13.3333,2.0316,0.9782,1.0122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.4833,4.9833,-1110.0000,47.4833,4.9833,-1110.0000,-1081.8000,-28.2000,0


In [ ]:
def estimate_lights_off_time(df, light_threshold=2):

    # 시간 → 실수형 (예: 23:30 → 23.5)
    def time_to_float(t):
        if pd.isna(t):
            return None
        return round(t.hour + t.minute / 60 + t.second / 3600, 4)

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    # 밤 시간대 필터 (21시~23시 or 0~3시)
    df['hour'] = df['timestamp'].dt.hour
    df = df[(df['hour'] >= 21) | (df['hour'] <= 3)].copy()

    # 자정 이후는 전날 night로 처리
    df.loc[df['hour'] <= 3, 'lifelog_date'] -= pd.Timedelta(days=1)

    # 낮은 조도 조건
    df = df[df['m_light'] <= light_threshold]

    # 각 (subject_id, lifelog_date)별 불 끈 시각 추출
    lights_off_df = (
        df.groupby(['subject_id', 'lifelog_date'])['timestamp']
        .min()
        .reset_index(name='lights_off_time')
    )

    # 실수형 시각으로 변환
    lights_off_df['lights_off_time'] = lights_off_df['lights_off_time'].dt.time.apply(time_to_float)

    return lights_off_df

In [ ]:
mLight23 = estimate_lights_off_time(mLight)
mLight23['lights_off_time'] = np.where(mLight23['lights_off_time']<24,mLight23['lights_off_time']+24,mLight23['lights_off_time'])
mLight23.head()

,subject_id,lifelog_date,lights_off_time
0,id01,2024-06-26,45.0500
1,id01,2024-06-27,45.1500
2,id01,2024-06-28,47.1500
3,id01,2024-06-29,45.9833
4,id01,2024-06-30,45.1500


### 🔥 mScreenStatus 화면 사용여부

- Indicates whether the smartphone screen is in use.
 - 기상시간, 취침시간, 수면시간
 - 휴대폰 이용횟수, 이용시간
 - 00 - 05 사이에 휴대폰 이용한 건수
 - 결측치 처리 x

In [ ]:
mScreenStatus['lifelog_date'] = mScreenStatus['timestamp'].astype(str).str[:10]
# mScreenStatus = fill_missing_dates_by_subject(mScreenStatus)
mScreenStatus.head(1)

,subject_id,timestamp,m_screen_use,lifelog_date
0,id01,2024-06-26 12:03:00,0,2024-06-26


In [ ]:
def preprocess_mScreenStatus(df):
    from datetime import datetime, time as dtime, timedelta

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    # base key 확보
    base_keys = df[['subject_id', 'lifelog_date']].drop_duplicates()
    base_keys['lifelog_date'] = base_keys['lifelog_date'].dt.date

    # 밤 9시부터 다음날 오전 11시 필터링
    df['hour'] = df['timestamp'].dt.hour
    df = df[(df['hour'] >= 21) | (df['hour'] < 11)].copy()
    df.loc[df['hour'] < 11, 'lifelog_date'] -= pd.Timedelta(days=1)

    df.sort_values(['subject_id', 'timestamp'], inplace=True)

    results = []

    for (subject_id, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        group = group.sort_values('timestamp').reset_index(drop=True)

        # 1. 중간 각성(앞뒤 0, 본인 1) 제거
        prev = group['m_screen_use'].shift(1)
        next_ = group['m_screen_use'].shift(-1)
        mask = (group['m_screen_use'] == 1) & (prev == 0) & (next_ == 0)
        group.loc[mask, 'm_screen_use'] = 0

        # 2. 블록 단위로 짧은 각성 블록 제거
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        block_info = group.groupby('block').agg(
            is_sleep=('is_sleep', 'first'),
            size=('is_sleep', 'size')
        )

        for i in range(1, len(block_info) - 1):
            if (
                block_info.iloc[i]['is_sleep'] == False and
                block_info.iloc[i]['size'] <= 2 and
                block_info.iloc[i - 1]['is_sleep'] and
                block_info.iloc[i + 1]['is_sleep']
            ):
                group.loc[group['block'] == block_info.index[i], 'm_screen_use'] = 0

        # 다시 블록 재계산 후 수면 추정
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        sleep_blocks = group[group['is_sleep']].groupby('block').agg(
            sleep_start=('timestamp', 'first'),
            sleep_end=('timestamp', 'last'),
            duration_min=('timestamp', lambda x: (x.max() - x.min()).total_seconds() / 60)
        )

        sleep_time = wake_time = duration_min = None
        if not sleep_blocks.empty:
            longest_sleep = sleep_blocks.loc[sleep_blocks['duration_min'].idxmax()]
            sleep_time = longest_sleep['sleep_start'].time()
            wake_time = longest_sleep['sleep_end'].time()
            duration_min = (
                datetime.combine(datetime.today(), wake_time) - datetime.combine(datetime.today(), sleep_time)
            ).total_seconds() / 60
            if duration_min < 0:
                duration_min += 1440

            if not (4 <= wake_time.hour < 11):
                wake_time = None
            if not (sleep_time.hour >= 21 or sleep_time.hour < 3):
                sleep_time = None
            if duration_min < 100:
                sleep_time = None
                wake_time = None
                duration_min = None

        results.append({
            'subject_id': subject_id,
            'lifelog_date': lifelog_date.date(),
            'sleep_time': sleep_time,
            'wake_time': wake_time,
            'sleep_duration_min': round(duration_min, 1) if duration_min is not None else None
        })


    sleep_df = pd.DataFrame(results)
    result_df = base_keys.merge(sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    # 시간 → 실수형 숫자 (예: 23:30 → 23.5)
    def time_to_float(t):
        if pd.isna(t):
            return None
        return round(t.hour + t.minute / 60 + t.second / 3600, 4)

    result_df['sleep_time'] = result_df['sleep_time'].apply(time_to_float)
    result_df['wake_time'] = result_df['wake_time'].apply(time_to_float)

    return result_df

In [ ]:
def preprocess_mScreenStatus(df):
    from datetime import datetime, timedelta

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    base_keys = df[['subject_id', 'lifelog_date']].drop_duplicates()
    base_keys['lifelog_date'] = base_keys['lifelog_date'].dt.date

    # 밤 9시 ~ 다음날 오전 11시 필터링
    df['hour'] = df['timestamp'].dt.hour
    df = df[(df['hour'] >= 21) | (df['hour'] < 11)].copy()
    df.loc[df['hour'] < 11, 'lifelog_date'] -= pd.Timedelta(days=1)
    df.sort_values(['subject_id', 'timestamp'], inplace=True)

    results = []

    for (subject_id, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        group = group.sort_values('timestamp').reset_index(drop=True)

        # 중간 각성 제거
        prev = group['m_screen_use'].shift(1)
        next_ = group['m_screen_use'].shift(-1)
        mask = (group['m_screen_use'] == 1) & (prev == 0) & (next_ == 0)
        group.loc[mask, 'm_screen_use'] = 0

        # 짧은 각성 블록 제거
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        block_info = group.groupby('block').agg(
            is_sleep=('is_sleep', 'first'),
            size=('is_sleep', 'size')
        )

        for i in range(1, len(block_info) - 1):
            if (
                block_info.iloc[i]['is_sleep'] == False and
                block_info.iloc[i]['size'] <= 2 and
                block_info.iloc[i - 1]['is_sleep'] and
                block_info.iloc[i + 1]['is_sleep']
            ):
                group.loc[group['block'] == block_info.index[i], 'm_screen_use'] = 0

        # 블록 재계산
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()

        sleep_blocks = group[group['is_sleep']].groupby('block').agg(
            sleep_start=('timestamp', 'first'),
            sleep_end=('timestamp', 'last'),
            duration_min=('timestamp', lambda x: (x.max() - x.min()).total_seconds() / 60)
        )

        sleep_time = wake_time = duration_min = None
        if not sleep_blocks.empty:
            longest_sleep = sleep_blocks.loc[sleep_blocks['duration_min'].idxmax()]
            sleep_time = longest_sleep['sleep_start'].time()
            wake_time = longest_sleep['sleep_end'].time()
            duration_min = longest_sleep['duration_min']  # ✅ 정확하게 자정 넘는 경우도 반영됨

            # 유효 시간 범위 조건
            if not (4 <= wake_time.hour < 11):
                wake_time = None
            if not (sleep_time.hour >= 21 or sleep_time.hour < 3):
                sleep_time = None
            if duration_min < 100:
                sleep_time = None
                wake_time = None
                duration_min = None

        results.append({
            'subject_id': subject_id,
            'lifelog_date': lifelog_date.date(),
            'sleep_time': sleep_time,
            'wake_time': wake_time,
            'sleep_duration_min': round(duration_min, 1) if duration_min is not None else None
        })

    sleep_df = pd.DataFrame(results)
    result_df = base_keys.merge(sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    # 시간 → 실수형 숫자 변환
    def time_to_float(t):
        if pd.isna(t):
            return None
        return round(t.hour + t.minute / 60 + t.second / 3600, 4)

    result_df['sleep_time'] = result_df['sleep_time'].apply(time_to_float)
    result_df['wake_time'] = result_df['wake_time'].apply(time_to_float)

    return result_df

In [ ]:
def add_ratios(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['weekday'] = df['lifelog_date'].dt.weekday  # 0=월 ~ 6=일
    df['week_type'] = df['weekday'].apply(lambda x: 'weekend' if x >= 5 else 'weekday')
    df['month'] = df['lifelog_date'].dt.month

    # 평균 계산
    avg_duration = df.groupby(['subject_id', 'month', 'week_type'])['sleep_duration_min'].mean().reset_index(name='avg_sleep_duration')
    sleep_time_avg = df.groupby(['subject_id', 'month', 'week_type'])['sleep_time'].apply(calculate_circular_mean_sleep_time).reset_index(name='avg_sleep_time')
    wake_time_avg = df.groupby(['subject_id', 'month', 'week_type'])['wake_time'].apply(calculate_circular_mean_sleep_time).reset_index(name='avg_wake_time')
    avg_df = sleep_time_avg.merge(wake_time_avg, on=['subject_id', 'month', 'week_type']).merge(avg_duration, on=['subject_id', 'month', 'week_type'])
    df = df.merge(avg_df, on=['subject_id', 'month', 'week_type'], how='left')

    # 비율 변수
    df['sleep_time_diff'] = df['avg_sleep_time'] - df['sleep_time']
    df['wake_time_diff'] = df['avg_wake_time'] - df['wake_time']
    df['sleep_duration_diff'] = df['avg_sleep_duration'] - df['sleep_duration_min']
    df['sleep_time_ratio'] = df['sleep_time'] / df['avg_sleep_time']
    df['wake_time_ratio'] = df['wake_time'] / df['avg_wake_time']
    df['sleep_duration_ratio'] = df['sleep_duration_min'] / df['avg_sleep_duration']

    # 정렬 및 lag/변화량
    df = df.sort_values(['subject_id', 'lifelog_date'])
    for lag in [1, 2]:
        df[f'sleep_time_lag{lag}'] = df.groupby('subject_id')['sleep_time'].shift(lag)
        df[f'wake_time_lag{lag}'] = df.groupby('subject_id')['wake_time'].shift(lag)
        df[f'sleep_duration_lag{lag}'] = df.groupby('subject_id')['sleep_duration_min'].shift(lag)
        df[f'sleep_time_diff_lag{lag}'] = df.groupby('subject_id')['sleep_time'].diff(lag)
        df[f'wake_time_diff_lag{lag}'] = df.groupby('subject_id')['wake_time'].diff(lag)
        df[f'sleep_duration_diff_lag{lag}'] = df.groupby('subject_id')['sleep_duration_min'].diff(lag)
    df['week_type_lag1'] = df.groupby('subject_id')['week_type'].shift(1)

    # 이동 평균
    for window in [2, 3]:
        df[f'rolling_sleep_time_{window}d'] = df.groupby('subject_id')['sleep_time'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)
        df[f'rolling_wake_time_{window}d'] = df.groupby('subject_id')['wake_time'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)
        df[f'rolling_sleep_duration_{window}d'] = df.groupby('subject_id')['sleep_duration_min'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)

    # 요일별 평균 수면 비교
    weekday_avg = df.groupby(['subject_id', 'weekday'])['sleep_duration_min'].mean().reset_index(name='weekday_avg_sleep')
    df = df.merge(weekday_avg, on=['subject_id', 'weekday'], how='left')
    df['sleep_duration_vs_weekday_avg'] = df['sleep_duration_min'] - df['weekday_avg_sleep']

    # 급격한 수면시간 변화 여부
    df['is_sleep_duration_change_large'] = (df['sleep_duration_diff_lag1'].abs() > 60).astype(int)

    return df

def calculate_circular_mean_sleep_time(series):
    radians = series * 2 * np.pi / 1440  # 분 단위 -> 라디안
    sin_sum = np.sin(radians).sum()
    cos_sum = np.cos(radians).sum()
    mean_angle = np.arctan2(sin_sum, cos_sum)
    if mean_angle < 0:
        mean_angle += 2 * np.pi
    return mean_angle * 1440 / (2 * np.pi)

In [ ]:
mScreenStatus2 = preprocess_mScreenStatus(mScreenStatus)
mScreenStatus2['sleep_time'] = np.where(mScreenStatus2['sleep_time']<24,mScreenStatus2['sleep_time']+24,mScreenStatus2['sleep_time'])
mScreenStatus2['sleep_duration_min'] = mScreenStatus2.apply(lambda x: calculate_sleep_duration_min(x['sleep_time'],x['wake_time']),axis=1)
mScreenStatus2 = add_ratios(mScreenStatus2)

# check
print(f'\n # mScreenStatus2 shape: {mScreenStatus2.shape}')
mScreenStatus2.head(1)


 # mScreenStatus2 shape: (700, 39)


,subject_id,lifelog_date,sleep_time,wake_time,sleep_duration_min,weekday,week_type,month,avg_sleep_time,avg_wake_time,avg_sleep_duration,sleep_time_diff,wake_time_diff,sleep_duration_diff,sleep_time_ratio,wake_time_ratio,sleep_duration_ratio,sleep_time_lag1,wake_time_lag1,sleep_duration_lag1,sleep_time_diff_lag1,wake_time_diff_lag1,sleep_duration_diff_lag1,sleep_time_lag2,wake_time_lag2,sleep_duration_lag2,sleep_time_diff_lag2,wake_time_diff_lag2,sleep_duration_diff_lag2,week_type_lag1,rolling_sleep_time_2d,rolling_wake_time_2d,rolling_sleep_duration_2d,rolling_sleep_time_3d,rolling_wake_time_3d,rolling_sleep_duration_3d,weekday_avg_sleep,sleep_duration_vs_weekday_avg,is_sleep_duration_change_large
0,id01,2024-06-26,47.4500,5.2500,-1092.0000,2,weekday,6,47.1944,5.4889,-1062.3333,-0.2556,0.2389,29.6667,1.0054,0.9565,1.0279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.4500,5.2500,-1092.0000,47.4500,5.2500,-1092.0000,-1063.0000,-29.0000,0


In [ ]:
mScreenStatus2평균수면시간 = mScreenStatus2.groupby(['subject_id','week_type']).apply(lambda x: pd.Series({
     '평균 취침시간':circular_mean_sleep_time(x['sleep_time'])
    ,'평균 기상시간':circular_mean_sleep_time(x['wake_time'])
    ,'평균 수면시간':x['sleep_duration_min'].mean()
})).reset_index()

# 저장
fname = f'{path}mScreenStatus2평균수면시간.xlsx'
print(fname)
mScreenStatus2평균수면시간.to_excel(fname, index=False)

# check
mScreenStatus2평균수면시간.head()

/content/drive/MyDrive/data/ch2025_data_items/mScreenStatus2평균수면시간.xlsx


,subject_id,week_type,평균 취침시간,평균 기상시간,평균 수면시간
0,id01,weekday,22:46,05:54,-937.6486
1,id01,weekend,22:25,06:04,-983.2941
2,id02,weekday,22:57,07:14,-563.4222
3,id02,weekend,23:13,07:27,-525.0417
4,id03,weekday,00:30,09:08,14.4286


### 🔥 mUsageStats 앱사용통계
- mUsageStats: Indicates which apps were used on the smartphone and for how long.

 - 몇시까지 핸드폰 보다가 잠잤는지
 - 통화, 전화 얼마나 했는지
 - YouTube 얼마나 봤는지
 - 메시지, 카카오톡 얼마나 했는지
 - NAVER 얼마나 했는지
 - 평소보다 얼마나 많은 앱을 이용했는지
 - 제외? -> 시스템 UI,One UI 홈

In [ ]:
def extract_mUsageStats_info(row):
    m_data = row['m_usage_stats']
    app_name = [item['app_name'] for item in m_data]
    total_time = [item['total_time'] for item in m_data]
    return pd.Series({'app_name': app_name, 'total_time': total_time})

mUsageStats[['app_name', 'total_time']] = mUsageStats.apply(extract_mUsageStats_info, axis=1)
mUsageStats['lifelog_date'] = mUsageStats['timestamp'].astype(str).str[:10]
# mUsageStats = fill_missing_dates_by_subject(mUsageStats)
mUsageStats.head(1)

,subject_id,timestamp,m_usage_stats,app_name,total_time,lifelog_date
0,id01,2024-06-26 13:00:00,"[{'app_name': ' 캐시워크', 'total_time': 69}, {'app_name': 'NAVER', 'total_time': 549}, {'app_name': ' ✝️성경일독Q', 'total_time': 7337}]","[ 캐시워크, NAVER, ✝️성경일독Q]","[69, 549, 7337]",2024-06-26


In [ ]:
def process_mUsageStats(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['요일'] = df['lifelog_date'].dt.day_name()

    # 리스트 평탄화
    exploded_df = df.explode(['app_name', 'total_time'])
    exploded_df['total_time'] = exploded_df['total_time'].astype(float)
    exploded_df['total_time'] = exploded_df['total_time'] * 0.001 / 60  # 밀리초 → 초 → 분 변환

    # app_name 특수문자 제거
    exploded_df['app_name'] = exploded_df['app_name'].astype(str).apply(
        lambda x: re.sub(r'[^가-힣a-zA-Z0-9]', '', x)
    )

    # 시스템 앱 제거
    filtered_df = exploded_df[~exploded_df['app_name'].isin(['시스템UI'])]  # '시스템UI'만 제거 (OneUI홈은 포함)

    # 주요 파생변수 생성
    def calculate_daily_metrics(group):
        last_use = group['timestamp'].max()

        app_times = {
            '통화_time': group[group['app_name'] == '통화']['total_time'].sum(),
            '전화_time': group[group['app_name'] == '전화']['total_time'].sum(),
            'YouTube_time': group[group['app_name'] == 'YouTube']['total_time'].sum(),
            '메신저_time': group[group['app_name'].isin(['메시지', '카카오톡'])]['total_time'].sum(),
            'NAVER_time': group[group['app_name'] == 'NAVER']['total_time'].sum(),
            '캐시워크_time': group[group['app_name'] == '캐시워크']['total_time'].sum(),
            '성경일독Q_time': group[group['app_name'] == '성경일독Q']['total_time'].sum(),
            'OneUI홈_time': group[group['app_name'] == 'OneUI홈']['total_time'].sum(),
        }

        return pd.Series({
            **app_times,
            'unique_app_count': group['app_name'].nunique(),
            'total_screen_time': group['total_time'].sum()
        })

    # daily metrics 생성
    daily_stats = filtered_df.groupby(['subject_id','lifelog_date']).apply(calculate_daily_metrics).reset_index()

    # subject_id별 평균 총화면시간 구하기
    avg_screen_time = daily_stats.groupby('subject_id')['total_screen_time'].mean().to_dict()

    # 평균대비 화면사용량(%) 생성
    def compute_screen_usage(row):
        avg_time = avg_screen_time.get(row['subject_id'], np.nan)
        if pd.isna(avg_time) or avg_time == 0:
            return np.nan
        return round((row['total_screen_time'] / avg_time - 1) * 100, 1)

    daily_stats['screen_time_vs_avg_pct'] = daily_stats.apply(compute_screen_usage, axis=1)

    return daily_stats

In [ ]:
def process_mUsageStats(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['요일'] = df['lifelog_date'].dt.day_name()
    df['hour'] = df['timestamp'].dt.hour

    # 시간대 분류
    def map_time_period(row):
        if 20 <= row['hour'] <= 23:
            return 'beforebed'
        else:
            return 'activehour'

    df['time_period'] = df.apply(map_time_period, axis=1)

    # 리스트 평탄화
    exploded_df = df.explode(['app_name', 'total_time'])
    exploded_df['total_time'] = exploded_df['total_time'].astype(float)
    exploded_df['total_time'] = exploded_df['total_time'] * 0.001 / 60  # 밀리초 → 초 → 분 변환

    # app_name 특수문자 제거
    exploded_df['app_name'] = exploded_df['app_name'].astype(str).apply(
        lambda x: re.sub(r'[^가-힣a-zA-Z0-9]', '', x)
    )

    # 시스템 앱 제거
    filtered_df = exploded_df[~exploded_df['app_name'].isin(['시스템UI'])]

    # 주요 파생변수 생성
    def calculate_daily_metrics(group):
        app_times = {
            '통화_time': group[group['app_name'] == '통화']['total_time'].sum(),
            '전화_time': group[group['app_name'] == '전화']['total_time'].sum(),
            'YouTube_time': group[group['app_name'] == 'YouTube']['total_time'].sum(),
            '메신저_time': group[group['app_name'].isin(['메시지', '카카오톡'])]['total_time'].sum(),
            'NAVER_time': group[group['app_name'] == 'NAVER']['total_time'].sum(),
            '캐시워크_time': group[group['app_name'] == '캐시워크']['total_time'].sum(),
            '성경일독Q_time': group[group['app_name'] == '성경일독Q']['total_time'].sum(),
            'OneUI홈_time': group[group['app_name'] == 'OneUI홈']['total_time'].sum(),
        }

        return pd.Series({
            **app_times,
            'unique_app_count': group['app_name'].nunique(),
            'total_screen_time': group['total_time'].sum()
        })

    # 일자/시간대별 요약
    daily_stats = filtered_df.groupby(['subject_id', 'lifelog_date', 'time_period']).apply(calculate_daily_metrics).reset_index()

    # subject_id별 평균 총화면시간
    avg_screen_time = daily_stats.groupby('subject_id')['total_screen_time'].mean().to_dict()

    # 평균 대비 비율
    def compute_screen_usage(row):
        avg_time = avg_screen_time.get(row['subject_id'], np.nan)
        if pd.isna(avg_time) or avg_time == 0:
            return np.nan
        return round((row['total_screen_time'] / avg_time - 1) * 100, 1)

    daily_stats['screen_time_vs_avg_pct'] = daily_stats.apply(compute_screen_usage, axis=1)

    # 피벗
    daily_stats = daily_stats.pivot(index=['subject_id', 'lifelog_date'], columns='time_period')
    daily_stats.columns = [f"{tp}_{metric}" for metric, tp in daily_stats.columns]
    daily_stats = daily_stats.reset_index()

    return daily_stats

In [ ]:
mUsageStats2 = process_mUsageStats(mUsageStats)

# check
print(f'\n # mUsageStats2 shape: {mUsageStats2.shape}')
mUsageStats2.head(1)


 # mUsageStats2 shape: (689, 24)


,subject_id,lifelog_date,activehour_통화_time,beforebed_통화_time,activehour_전화_time,beforebed_전화_time,activehour_YouTube_time,beforebed_YouTube_time,activehour_메신저_time,beforebed_메신저_time,activehour_NAVER_time,beforebed_NAVER_time,activehour_캐시워크_time,beforebed_캐시워크_time,activehour_성경일독Q_time,beforebed_성경일독Q_time,activehour_OneUI홈_time,beforebed_OneUI홈_time,activehour_unique_app_count,beforebed_unique_app_count,activehour_total_screen_time,beforebed_total_screen_time,activehour_screen_time_vs_avg_pct,beforebed_screen_time_vs_avg_pct
0,id01,2024-06-26,9.0010,0.2079,11.3007,0.7731,0.1061,0.0000,43.6359,14.5713,8.4852,0.1351,18.6694,5.4722,88.3836,27.6892,61.1160,27.9861,25.0000,20.0000,266.7672,156.8681,-29.0000,-58.3000


### ✔️ mWifi 주변wifi (수정)
- Wifi devices around individual subject.
 - -30 ~ -50 dBm	매우 강한 신호 (최적)
 - -51 ~ -60 dBm	강한 신호 (문제 없음)
 - -61 ~ -70 dBm	괜찮은 신호 (약간 느릴 수 있음)
 - -71 ~ -80 dBm	약한 신호 (끊김 주의)
 - -81 dBm 이하	매우 약한 신호 (거의 끊김)

In [ ]:
def extract_wifi_info(row):
    wifi_data = row['m_wifi']
    bssids = [item['bssid'] for item in wifi_data]
    rssis = [item['rssi'] for item in wifi_data]
    return pd.Series({'bssid': bssids, 'rssi': rssis})

mWifi = pd.read_parquet(path+'ch2025_mWifi.parquet')
mWifi[['bssid', 'rssi']] = mWifi.apply(extract_wifi_info, axis=1)
mWifi['lifelog_date'] = mWifi['timestamp'].astype(str).str[:10]
# mWifi = fill_missing_dates_by_subject(mWifi)
mWifi.head(1)

,subject_id,timestamp,m_wifi,bssid,rssi,lifelog_date
0,id01,2024-06-26 12:03:00,"[{'bssid': 'a0:0f:37:9a:5d:8b', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8c', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8d', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8e', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8f', 'rssi': -78}, {'bssid': 'a0:0f:37:96:56:ef', 'rssi': -58}, {'bssid': '88:36:6c:86:75:84', 'rssi': -72}, {'bssid': 'a0:0f:37:96:56:ee', 'rssi': -58}, {'bssid': 'a0:0f:37:96:56:ed', 'rssi': -58}, {'bssid': '86:25:19:b5:b2:a5', 'rssi': -61}, {'bssid': 'a0:0f:37:96:56:ec', 'rssi': -58}, {'bssid': '1e:39:29:8e:fb:e9', 'rssi': -71}, {'bssid': '52:c2:e8:c7:9b:e4', 'rssi': -82}, {'bssid': 'a0:0f:37:96:56:eb', 'rssi': -58}, {'bssid': '12:e3:c7:09:20:34', 'rssi': -88}, {'bssid': '58:86:94:4a:08:b8', 'rssi': -82}, {'bssid': '90:9f:33:28:d0:2e', 'rssi': -78}, {'bssid': '00:26:66:bc:4e:18', 'rssi': -85}, {'bssid': 'f6:0a:f4:43:4b:ba', 'rssi': -45}, {'bssid': '10:e3:c7:09:20:35', 'rssi': -63}, {'bssid': '10:e3:c7:09:20:34', 'rssi': -89}, {'bssid': '1c:39:29:48:04:92', 'rssi': -82}, {'bssid': '12:e3:c7:07:9d:df', 'rssi': -83}, {'bssid': '86:25:19:c3:44:07', 'rssi': -84}, {'bssid': 'a0:0f:37:9a:37:2f', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2e', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2d', 'rssi': -76}, {'bssid': '0a:09:b4:74:05:ec', 'rssi': -72}, {'bssid': 'a0:0f:37:9a:37:2c', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2b', 'rssi': -76}, {'bssid': '0a:09:b4:74:05:eb', 'rssi': -59}, {'bssid': 'c0:25:2f:d8:c1:a6', 'rssi': -82}, {'bssid': '16:7f:67:bb:fa:f8', 'rssi': -79}, {'bssid': '3c:f3:92:ff:00:01', 'rssi': -82}, {'bssid': '06:09:b4:74:05:ec', 'rssi': -72}, {'bssid': '06:09:b4:74:05:eb', 'rssi': -59}, {'bssid': '12:e3:c7:0a:74:d1', 'rssi': -78}, {'bssid': '88:36:6c:a9:6f:8e', 'rssi': -63}, {'bssid': '02:e3:c7:09:20:34', 'rssi': -88}, {'bssid': '00:09:b4:74:05:eb', 'rssi': -60}, {'bssid': '00:09:b4:74:05:ec', 'rssi': -72}, {'bssid': '00:1d:93:93:cf:fe', 'rssi': -19}, {'bssid': '8e:e2:ac:a5:9d:15', 'rssi': -72}]","[a0:0f:37:9a:5d:8b, a0:0f:37:9a:5d:8c, a0:0f:37:9a:5d:8d, a0:0f:37:9a:5d:8e, a0:0f:37:9a:5d:8f, a0:0f:37:96:56:ef, 88:36:6c:86:75:84, a0:0f:37:96:56:ee, a0:0f:37:96:56:ed, 86:25:19:b5:b2:a5, a0:0f:37:96:56:ec, 1e:39:29:8e:fb:e9, 52:c2:e8:c7:9b:e4, a0:0f:37:96:56:eb, 12:e3:c7:09:20:34, 58:86:94:4a:08:b8, 90:9f:33:28:d0:2e, 00:26:66:bc:4e:18, f6:0a:f4:43:4b:ba, 10:e3:c7:09:20:35, 10:e3:c7:09:20:34, 1c:39:29:48:04:92, 12:e3:c7:07:9d:df, 86:25:19:c3:44:07, a0:0f:37:9a:37:2f, a0:0f:37:9a:37:2e, a0:0f:37:9a:37:2d, 0a:09:b4:74:05:ec, a0:0f:37:9a:37:2c, a0:0f:37:9a:37:2b, 0a:09:b4:74:05:eb, c0:25:2f:d8:c1:a6, 16:7f:67:bb:fa:f8, 3c:f3:92:ff:00:01, 06:09:b4:74:05:ec, 06:09:b4:74:05:eb, 12:e3:c7:0a:74:d1, 88:36:6c:a9:6f:8e, 02:e3:c7:09:20:34, 00:09:b4:74:05:eb, 00:09:b4:74:05:ec, 00:1d:93:93:cf:fe, 8e:e2:ac:a5:9d:15]","[-78, -78, -78, -78, -78, -58, -72, -58, -58, -61, -58, -71, -82, -58, -88, -82, -78, -85, -45, -63, -89, -82, -83, -84, -76, -76, -76, -72, -76, -76, -59, -82, -79, -82, -72, -59, -78, -63, -88, -60, -72, -19, -72]",2024-06-26


In [ ]:
def process_mWifi(df,threshold):

    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['요일'] = df['lifelog_date'].dt.day_name()
    df['hour'] = df['timestamp'].dt.hour

    # 시간대 분류
    def map_time_period(row):
        if 20 <= row['hour'] <= 23:
            return 'beforebed'
        else:
            return 'activehour'

    df['time_period'] = df.apply(map_time_period, axis=1)

    features = []
    grouped = df.groupby(['subject_id', 'lifelog_date','time_period'])

    for (subject_id, date, period), group in grouped:
        scan_count = len(group)
        bssid_flat = sum(group['bssid'], [])  # flatten
        rssi_flat = sum(group['rssi'], [])    # flatten

        unique_bssid_count = len(set(bssid_flat))
        avg_rssi = sum(rssi_flat) / len(rssi_flat) if rssi_flat else None
        max_rssi = max(rssi_flat) if rssi_flat else None
        min_rssi = min(rssi_flat) if rssi_flat else None
        strong_rssi_ratio = sum(1 for r in rssi_flat if r > -60) / len(rssi_flat) if rssi_flat else 0
        empty_scan_count = sum(1 for b in group['bssid'] if len(b) == 0)

        # 가장 많이 탐지된 BSSID
        bssid_counter = Counter(bssid_flat)
        top_bssid, top_bssid_count = bssid_counter.most_common(1)[0] if bssid_counter else (None, 0)

        first_time = group['timestamp'].min()
        last_time = group['timestamp'].max()
        hour_span = (last_time - first_time).total_seconds() / 60  # 분 단위

        features.append({
            'subject_id': subject_id,
            'lifelog_date': date,
            'time_period': period,  #
            'scan_count': scan_count,
            'unique_bssid_count': unique_bssid_count,
            'avg_rssi': avg_rssi,
            'max_rssi': max_rssi,
            'min_rssi': min_rssi,
            'strong_signal_ratio': strong_rssi_ratio,
            'empty_scan_count': empty_scan_count,
            'top_bssid': top_bssid,
            'top_bssid_count': top_bssid_count,
            'hour_span_minutes': hour_span
        })

    daily_stats = pd.DataFrame(features)

    # 피벗
    daily_stats = daily_stats.pivot(index=['subject_id', 'lifelog_date'], columns='time_period')
    daily_stats.columns = [f"{tp}_{metric}" for metric, tp in daily_stats.columns]
    daily_stats = daily_stats.reset_index()

    return daily_stats

In [ ]:
mWifi2 = process_mWifi(mWifi,threshold=-60)

# check
print(f'\n # mWifi2 shape: {mWifi2.shape}')
mWifi2.head(1)


 # mWifi2 shape: (685, 22)


,subject_id,lifelog_date,activehour_scan_count,beforebed_scan_count,activehour_unique_bssid_count,beforebed_unique_bssid_count,activehour_avg_rssi,beforebed_avg_rssi,activehour_max_rssi,beforebed_max_rssi,activehour_min_rssi,beforebed_min_rssi,activehour_strong_signal_ratio,beforebed_strong_signal_ratio,activehour_empty_scan_count,beforebed_empty_scan_count,activehour_top_bssid,beforebed_top_bssid,activehour_top_bssid_count,beforebed_top_bssid_count,activehour_hour_span_minutes,beforebed_hour_span_minutes
0,id01,2024-06-26,48.0000,21.0000,354.0000,56.0000,-71.7103,-64.5025,-19.0000,-26.0000,-91.0000,-88.0000,0.1892,0.3881,0.0000,0.0000,86:25:19:9f:9b:be,04:09:a5:3a:c8:6a,19.0000,15.0000,470.0000,236.0000


### ✔️ wHr 심박동수 (수정)
- Heart rate readings recorded by the smartwatch.


In [ ]:
wHr['lifelog_date'] = wHr['timestamp'].astype(str).str[:10]
# wHr = fill_missing_dates_by_subject(wHr)
wHr.head(1)

,subject_id,timestamp,heart_rate,lifelog_date
0,id01,2024-06-26 12:23:00,"[134, 134, 135, 133, 134, 135, 134, 135, 134, 133, 133, 133, 132, 132, 131, 131, 131, 132, 132, 134, 134, 134, 132, 130, 128, 126, 126, 126, 127, 129, 130, 129, 130, 130, 127, 127, 126, 125, 123]",2024-06-26


In [ ]:
def process_wHr(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['hour'] = df['timestamp'].dt.hour
    df['block'] = df['hour'].map(get_time_block)

    # lifelog_date 정의 시 sleeptime은 하루 차감
    df['lifelog_date'] = df['timestamp'].dt.date
    df.loc[df['block'] == 'sleeptime', 'lifelog_date'] = \
        df.loc[df['block'] == 'sleeptime', 'lifelog_date'] - pd.Timedelta(days=1)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'lifelog_date']):
        block_stats = {'subject_id': subj, 'lifelog_date': date}

        for block, block_group in group.groupby('block'):
            hr_all = []
            for row in block_group['heart_rate']:
                parsed = ast.literal_eval(row) if isinstance(row, str) else row
                hr_all.extend([int(h) for h in parsed if h is not None])

            if not hr_all:
                continue

            above_100 = [hr for hr in hr_all if hr > 100]
            block_stats[f'hr_{block}_mean'] = np.mean(hr_all)
            block_stats[f'hr_{block}_std'] = np.std(hr_all)
            block_stats[f'hr_{block}_max'] = np.max(hr_all)
            block_stats[f'hr_{block}_min'] = np.min(hr_all)
            block_stats[f'hr_{block}_above_100_ratio'] = len(above_100) / len(hr_all)

        results.append(block_stats)

    return pd.DataFrame(results)

In [ ]:
wHr2 = process_wHr(wHr)

# check
print(f'\n # wHr2 shape: {wHr2.shape}')
wHr2.head(1)


 # wHr2 shape: (669, 12)


,subject_id,lifelog_date,hr_activehour_mean,hr_activehour_std,hr_activehour_max,hr_activehour_min,hr_activehour_above_100_ratio,hr_sleeptime_mean,hr_sleeptime_std,hr_sleeptime_max,hr_sleeptime_min,hr_sleeptime_above_100_ratio
0,id01,2024-06-26,81.2434,11.8712,142.0000,59.0000,0.0693,NaN,NaN,NaN,NaN,NaN


### ✔️ wLight 라이트 (수정)
- Ambient light measured by the smartwatch.  
  - 어두운 밤 0.1 ~ 1 lux 캄캄한 방, 달빛 없는 밤
  - 가로등 켜진 거리 10 ~ 20 lux 흐릿한 외부 조명
  - 실내 조명 100 ~ 500 lux 사무실, 일반 거실
  - 밝은 실외 10,000 ~ 25,000 lux 맑은 날 햇빛
  - 직사광선 아래 30,000 ~ 100,000 lux 여름 한낮, 매우 강한 햇빛

In [ ]:
wLight['lifelog_date'] = wLight['timestamp'].astype(str).str[:10]
# wLight = fill_missing_dates_by_subject(wLight)
wLight.head(1)

,subject_id,timestamp,w_light,lifelog_date
0,id01,2024-06-26 12:17:00,633.0000,2024-06-26


In [ ]:
def process_wLight(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['hour'] = df['timestamp'].dt.hour
    df['block'] = df['hour'].map(get_time_block)

    # 기본 lifelog_date
    df['lifelog_date'] = df['timestamp'].dt.date
    # sleeptime에 대해 하루 차감
    df.loc[df['block'] == 'sleeptime', 'lifelog_date'] = \
        df.loc[df['block'] == 'sleeptime', 'lifelog_date'] - pd.Timedelta(days=1)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'lifelog_date']):
        block_stats = {'subject_id': subj, 'lifelog_date': date}

        for block, block_group in group.groupby('block'):
            lux = block_group['w_light'].dropna().values
            if len(lux) == 0:
                continue

            block_stats[f'wlight_{block}_mean'] = np.mean(lux)
            block_stats[f'wlight_{block}_std'] = np.std(lux)
            block_stats[f'wlight_{block}_max'] = np.max(lux)
            block_stats[f'wlight_{block}_min'] = np.min(lux)

        results.append(block_stats)

    return pd.DataFrame(results)

In [ ]:
wLight2 = process_wLight(wLight)

# check
print(f'\n # wLight2 shape: {wLight2.shape}')
wLight2.head(1)


 # wLight2 shape: (755, 10)


,subject_id,lifelog_date,wlight_activehour_mean,wlight_activehour_std,wlight_activehour_max,wlight_activehour_min,wlight_sleeptime_mean,wlight_sleeptime_std,wlight_sleeptime_max,wlight_sleeptime_min
0,id01,2024-06-26,299.4151,1220.1126,20874.0000,0.0000,0.0000,0.0000,0.0000,0.0000


### ✔️ wPedo 걸음수
- Step data recorded by the smartwatch.

In [ ]:
wPedo['lifelog_date'] = wPedo['timestamp'].astype(str).str[:10]
# wPedo = fill_missing_dates_by_subject(wPedo)
wPedo.head(1)

,subject_id,timestamp,step,step_frequency,running_step,walking_step,distance,speed,burned_calories,lifelog_date
0,id01,2024-06-26 12:09:00,10,0.1667,0,0,8.3300,0.1388,0.0000,2024-06-26


In [ ]:
def process_wPedo(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date

    summary = df.groupby(['subject_id', 'lifelog_date']).agg({
        'step': 'sum',
        'step_frequency': 'mean',
        'distance': 'sum',
        'speed': ['mean', 'max'],
        'burned_calories': 'sum'
    }).reset_index()

    # 컬럼 이름 정리
    summary.columns = ['subject_id', 'lifelog_date',
                       'step_sum', 'step_frequency_mean',
                       'distance_sum', 'speed_mean', 'speed_max',
                       'burned_calories_sum']

    return summary

In [ ]:
wPedo2 = process_wPedo(wPedo)

# check
print(f'\n # wPedo2 shape: {wPedo2.shape}')
wPedo2.head(1)


 # wPedo2 shape: (653, 8)


,subject_id,lifelog_date,step_sum,step_frequency_mean,distance_sum,speed_mean,speed_max,burned_calories_sum
0,id01,2024-06-26,3578,0.0927,2782.1901,0.0721,1.5882,189.3191


### 🔥 운동 추정 파생변수

- mActivity 추정행동
- mGps, 핸드폰 위치
- wHr 심박동수
- wPedo 걸음수

In [ ]:
def average_list_columns(df, list_columns, pk_cols=['subject_id', 'lifelog_date']):

    for col in list_columns:

        def safe_mean(x):
            if isinstance(x, list):
                return np.mean(x) if len(x) > 0 else np.nan
            elif isinstance(x, (int, float, np.integer, np.floating, type(None))):
                return x
            elif isinstance(x, (np.ndarray, pd.Series)):
                return np.mean(x)
            elif pd.api.types.is_scalar(x) and pd.isna(x):
                return np.nan
            else:
                return np.nan

        df[col] = df[col].apply(safe_mean)

    return df

def compute_estimated_exercise(mActivity, mGps, wHr, wPedo, minutes):

    # 리스트 평균 처리
    mGps = mGps.copy()
    wHr = wHr.copy()
    mGps = average_list_columns(mGps, ['speed'])
    Hr = average_list_columns(wHr, ['heart_rate'])

    for df in [mActivity, mGps, wHr, wPedo]:
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # 5분 지속 조건 판단 함수
    def sustained_condition(df, cond_col,minutes):
        df = df[df[cond_col]].sort_values('timestamp')
        times = df['timestamp']
        start = prev = None
        for t in times:
            if start is None:
                start = prev = t
            elif t <= prev + timedelta(minutes=1):
                prev = t
            else:
                if prev - start >= timedelta(minutes=minutes):
                    return True
                start = prev = t
        return (prev - start) >= timedelta(minutes=minutes) if start else False

    # mActivity: m_activity == 7 지속
    mActivity['m_cond'] = mActivity['m_activity'] == 7
    act_flag = mActivity.groupby(['subject_id', 'lifelog_date']) \
                        .apply(lambda df: sustained_condition(df, 'm_cond',40)) \
                        .reset_index(name='act_exe_flag')

    # mGps: speed ∈ [2.5, 5.5] 지속
    mGps['gps_cond'] = mGps['speed'].between(2.5, 5.5)
    gps_flag = mGps.groupby(['subject_id', 'lifelog_date']) \
                   .apply(lambda df: sustained_condition(df, 'gps_cond',minutes)) \
                   .reset_index(name='gps_exe_flag')

    # wHr: hr ≥ 133 상태가 5분 이상 유지
    wHr['whr_cond'] = wHr['heart_rate'] >= 133
    hr_flag = wHr.groupby(['subject_id', 'lifelog_date']) \
                   .apply(lambda df: sustained_condition(df, 'whr_cond',minutes)) \
                   .reset_index(name='hr_exe_flag')

    # wPedo: step ≥ 10000 또는 running_step ≥ 1이 5분 이상
    pedo_daily = wPedo.groupby(['subject_id', 'lifelog_date'])['step'].sum().reset_index(name='total_steps')
    pedo_daily['step_flag'] = pedo_daily['total_steps'] >= 10000

    wPedo['r_cond'] = wPedo['running_step'] >= 1
    run_flag = wPedo.groupby(['subject_id', 'lifelog_date']) \
                    .apply(lambda df: sustained_condition(df, 'r_cond', minutes)) \
                    .reset_index(name='run_flag')

    pedo_flag = pedo_daily.merge(run_flag, on=['subject_id', 'lifelog_date'], how='outer')
    pedo_flag['step_flag'] = pedo_flag['step_flag'].fillna(False)
    pedo_flag['run_flag'] = pedo_flag['run_flag'].fillna(False)
    pedo_flag['pedo_exe_flag'] = pedo_flag[['step_flag', 'run_flag']].any(axis=1)

    # 병합 및 최종 판단
    result = act_flag.merge(gps_flag, on=['subject_id', 'lifelog_date'], how='outer') \
                     .merge(hr_flag, on=['subject_id', 'lifelog_date'], how='outer') \
                     .merge(pedo_flag[['subject_id', 'lifelog_date', 'pedo_exe_flag']], on=['subject_id', 'lifelog_date'], how='outer')

    # NaN 처리 및 1/0 변환
    for col in ['act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']:
        result[col] = result[col].fillna(False)

    result['pred_exe_flag'] = result[['act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']].any(axis=1)

    # 👉 1/0 변환
    for col in ['act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag', 'pred_exe_flag']:
        result[col] = result[col].astype(int)

    display(result[['pred_exe_flag', 'act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']].sum())

    return result[['subject_id', 'lifelog_date', 'pred_exe_flag', 'act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']]

In [ ]:
mActivity = pd.read_parquet(path+'ch2025_mActivity.parquet')
mActivity['lifelog_date'] = mActivity['timestamp'].astype(str).str[:10]

# 추정운동여부
exeFlag = compute_estimated_exercise(mActivity, mGps, wHr, wPedo,10)

# check
print(f'\n # exeFlag shape: {exeFlag.shape}')
exeFlag.head(1)

,0
pred_exe_flag,66
act_exe_flag,31
gps_exe_flag,5
hr_exe_flag,18
pedo_exe_flag,19



 # exeFlag shape: (700, 7)


,subject_id,lifelog_date,pred_exe_flag,act_exe_flag,gps_exe_flag,hr_exe_flag,pedo_exe_flag
0,id01,2024-06-26,0,0,0,0,0


### 🔥 Sleeptime 일어난 건수

- Sleeptime에 (mLight 주변 밝기), (wLight 앰비언트 라이트) 변화 건수

In [ ]:
def compute_night_awake_features(df, prefix):

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # 00시~06시 필터
    df['hour'] = df['timestamp'].dt.hour
    df_night = df[(df['hour'] >= 0) & (df['hour'] < 6)].copy()

    # 깨어있는 분 계산
    df_night['awake_minute'] = (df_night[prefix] > 0).astype(int)

    # 깨어난 횟수 계산 (0 → 양수 전환)
    def count_awake_blocks(x):
        return ((x > 0) & (x.shift(fill_value=0) == 0)).sum()

    # 그룹별 집계
    result = df_night.groupby(['subject_id', 'lifelog_date']).agg(
        awake_minutes=('awake_minute', 'sum'),
        awake_blocks=(prefix, count_awake_blocks)
    ).reset_index()

    # 컬럼명 변경
    result = result.rename(columns={
        'awake_minutes': f'{prefix}_awake_minutes',
        'awake_blocks': f'{prefix}_awake_blocks'
    })

    # train에 결과 합치기 위해서 -1 day 하기
    result['lifelog_date'] = pd.to_datetime(result['lifelog_date'])
    result['lifelog_date'] = result['lifelog_date'] + pd.Timedelta(days=-1)

    result['lifelog_date'] = result['lifelog_date'].astype(str)

    return result

In [ ]:
mAwakeBlocks = compute_night_awake_features(mLight,'m_light')
wAwakeBlocks = compute_night_awake_features(wLight,'w_light')
AwakeBlocks = mAwakeBlocks.merge(wAwakeBlocks, on=['subject_id','lifelog_date'], how='outer')
AwakeBlocks['light_awake_minutes'] = AwakeBlocks[['m_light_awake_minutes','w_light_awake_minutes']].max(axis=1)
AwakeBlocks['light_awake_blocks'] = AwakeBlocks[['m_light_awake_blocks','w_light_awake_blocks']].max(axis=1)

# check
print(mAwakeBlocks.shape)
print(wAwakeBlocks.shape)
print(AwakeBlocks.shape)

(677, 4)
(565, 4)
(687, 8)


### 📦 merge 데이터
- train, test 기간 서로 겹침

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/ch2025_metrics_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/ch2025_submission_sample.csv')

# 일자변수 타입 변환
mACStatus2['lifelog_date'] = mACStatus2['lifelog_date'].astype(str)
mActivity21['lifelog_date'] = mActivity21['lifelog_date'].astype(str)
mActivity22['lifelog_date'] = mActivity22['lifelog_date'].astype(str)
mAmbience2['lifelog_date'] = mAmbience2['lifelog_date'].astype(str)
mBle2['lifelog_date'] = mBle2['lifelog_date'].astype(str)
# mGps2['lifelog_date'] = mGps2['lifelog_date'].astype(str)
mLight21['lifelog_date'] = mLight21['lifelog_date'].astype(str)
mLight22['lifelog_date'] = mLight22['lifelog_date'].astype(str)
mLight23['lifelog_date'] = mLight23['lifelog_date'].astype(str)
mScreenStatus2['lifelog_date'] = mScreenStatus2['lifelog_date'].astype(str)
mUsageStats2['lifelog_date'] = mUsageStats2['lifelog_date'].astype(str)
mWifi2['lifelog_date'] = mWifi2['lifelog_date'].astype(str)
wHr2['lifelog_date'] = wHr2['lifelog_date'].astype(str)
wLight2['lifelog_date'] = wLight2['lifelog_date'].astype(str)
wPedo2['lifelog_date'] = wPedo2['lifelog_date'].astype(str)

# ---- new ----

exeFlag['lifelog_date'] = exeFlag['lifelog_date'].astype(str)
AwakeBlocks['lifelog_date'] = AwakeBlocks['lifelog_date'].astype(str)

In [ ]:
df_list = [
    mACStatus2,       # 1
    mActivity21,       # 2
    mActivity22,       # 2
    mAmbience2,       # 3
    mBle2,            # 4
    # mGps2,            # 5
    mLight21,          # 6
    mLight22,          # 6
    mLight23,          # 6
    mScreenStatus2,   # 7
    mUsageStats2,     # 8
    mWifi2,           # 9
    wHr2,             # 10
    wLight2,          # 11
    wPedo2,           # 12
    # ---- new ----
    # exeFlag,
    AwakeBlocks
]

data = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'lifelog_date'], how='outer'), df_list)
data['lifelog_date'] = data['lifelog_date'].astype(str)

# 중복체크
print(data.shape)
print(data[['subject_id','lifelog_date']].drop_duplicates().shape)

# merge
train2 = train.merge(data, on=['subject_id','lifelog_date'], how='left')
test2 = test.merge(data, on=['subject_id','lifelog_date'], how='left')

# 저장
print('# train  shape:',train.shape)
print('# train2 shape:',test2.shape)
print('# test   shape:',test.shape)
print('# test2  shape:',test2.shape)

(806, 203)
(806, 2)
# train  shape: (450, 9)
# train2 shape: (250, 210)
# test   shape: (250, 9)
# test2  shape: (250, 210)


In [ ]:
# 저장
train2.to_parquet(f"/content/drive/MyDrive/data/train_0520.parquet")
test2.to_parquet(f"/content/drive/MyDrive/data/test_0520.parquet")

### 📌 전처리 파일 불러오기

In [ ]:
train2 = pd.read_parquet(f"/content/drive/MyDrive/data/train_0520.parquet")
test2 = pd.read_parquet(f"/content/drive/MyDrive/data/test_0520.parquet")

In [ ]:
train = train2.copy()
test = test2.copy()

# drop_features = ['afterwork_max_label','sleeptime_max_label','worktime_max_label']
drop_features = ['top_bssid'] # ,'week_type','week_type_lag1'
drop_features = [i for i in drop_features if i in train.columns.tolist()]
print('# drop_features:',drop_features)
train = train.drop(columns=drop_features)
test = test.drop(columns=drop_features)

# drop_features: []


### 🔥 이미지 파생변수

In [ ]:
img_model = 'resnet50'

# 데이터 읽기
img_features = pd.read_csv(f'/content/drive/MyDrive/data/ch2025_data_items/img_features_ch5_sleeptime_{img_model}_10.csv')
img_features = img_features[sorted(img_features.columns,reverse=True)]
img_features.columns = ['image_path']+['img'+i for i in img_features.columns if i not in ['image_path']]

# 정규표현식으로 추출
img_features['subject_id'] = img_features['image_path'].str.extract(r'user(id\d+)_')[0]
img_features['lifelog_date'] = img_features['image_path'].str.extract(r'_(\d{4}-\d{2}-\d{2})_')[0]

# check
img_features = img_features.drop(columns=['image_path'])
print(len(img_features))
display(img_features.head(1))

# add img features
train['lifelog_date'] = train['lifelog_date'].astype(str)
test['lifelog_date'] = test['lifelog_date'].astype(str)
train = train.merge(img_features,on=['subject_id','lifelog_date'],how='left')
test = test.merge(img_features,on=['subject_id','lifelog_date'],how='left')

700


,img9,img8,img7,img6,img5,img4,img3,img2,img1,img0,subject_id,lifelog_date
0,-0.0646,-0.1787,-0.1334,-0.0200,-0.2189,0.2700,0.0377,0.2072,-0.4265,1.1548,id01,2024-06-26


### 🔥 추정수면효율
- 추정수면효율 (S2) : (불끈 시간 - 핸드폰 이용한 마지막 시간) / 추정수면시간

In [ ]:
def calculate_sleep_duration_min(sleep_time, wake_time):
    """
    취침 시각(sleep_time)과 기상 시각(wake_time)을 입력받아 수면 시간(분) 반환
    단위는 float 시간 (예: 23.5, 6.25)
    """
    if pd.isna(sleep_time) or pd.isna(wake_time):
        return None
    if wake_time < sleep_time:
        wake_time += 24  # 자정 넘긴 경우 보정
    duration = (wake_time - sleep_time) * 60
    return round(duration)

In [ ]:
train['불끈시간부터기상시간'] = train.apply(lambda x: calculate_sleep_duration_min(x['lights_off_time'],x['wake_time']),axis=1)
test['불끈시간부터기상시간'] = test.apply(lambda x: calculate_sleep_duration_min(x['lights_off_time'],x['wake_time']),axis=1)

In [ ]:
train['추정수면효율'] = train['불끈시간부터기상시간']/train['sleep_duration_min']
test['추정수면효율'] = test['불끈시간부터기상시간']/test['sleep_duration_min']

# 이상값 제거
train['추정수면효율'] = np.where(train['추정수면효율']<-5,np.nan,train['추정수면효율'])
test['추정수면효율'] = np.where(test['추정수면효율']<-5,np.nan,test['추정수면효율'])
train['추정수면효율'] = np.where(train['추정수면효율']>5,np.nan,train['추정수면효율'])
test['추정수면효율'] = np.where(test['추정수면효율']>55,np.nan,test['추정수면효율'])

In [ ]:
# train['추정수면효율2'] = train['불끈시간부터기상시간'] - train['sleep_duration_min']
# test['추정수면효율2'] = test['불끈시간부터기상시간'] - test['sleep_duration_min']

### 🔥 추가 파생변수

In [ ]:
# sleep duration

train['sleep_time_m_light_sleep_time'] = train['sleep_time'] - train['light_sleep_time']
test['sleep_time_m_light_sleep_time'] = test['sleep_time'] - test['light_sleep_time']

train['wake__time_m_light_wake__time'] = train['wake_time'] - train['light_wake_time']
test['wake__time_m_light_wake__time'] = test['wake_time'] - test['light_wake_time']

train['sleep_duration_min_m_light_sleep_duration_min'] = train['sleep_duration_min'] - train['light_sleep_duration_min']
test['sleep_duration_min_m_light_sleep_duration_min'] = test['sleep_duration_min'] - test['light_sleep_duration_min']

#

train['sleep_time_d_light_sleep_time'] = train['sleep_time'] / train['light_sleep_time']
test['sleep_time_d_light_sleep_time'] = test['sleep_time'] / test['light_sleep_time']

train['wake__time_d_light_wake__time'] = train['wake_time'] / train['light_wake_time']
test['wake__time_d_light_wake__time'] = test['wake_time'] / test['light_wake_time']

train['sleep_duration_min_d_light_sleep_duration_min'] = train['sleep_duration_min'] / train['light_sleep_duration_min']
test['sleep_duration_min_d_light_sleep_duration_min'] = test['sleep_duration_min'] / test['light_sleep_duration_min']

#

train['sleep_time_min'] = train[['sleep_time','light_sleep_time']].min(axis=1)
train['sleep_time_max'] = train[['sleep_time','light_sleep_time']].max(axis=1)

train['wake_time_min'] = train[['wake_time','light_wake_time']].min(axis=1)
train['wake_time_max'] = train[['wake_time','light_wake_time']].max(axis=1)

train['sleep_duration_min_min'] = train[['sleep_duration_min','light_sleep_duration_min']].min(axis=1)
train['sleep_duration_min_max'] = train[['sleep_duration_min','light_sleep_duration_min']].max(axis=1)

test['sleep_time_min'] = test[['sleep_time','light_sleep_time']].min(axis=1)
test['sleep_time_max'] = test[['sleep_time','light_sleep_time']].max(axis=1)

test['wake_time_min'] = test[['wake_time','light_wake_time']].min(axis=1)
test['wake_time_max'] = test[['wake_time','light_wake_time']].max(axis=1)

test['sleep_duration_min_min'] = test[['sleep_duration_min','light_sleep_duration_min']].min(axis=1)
test['sleep_duration_min_max'] = test[['sleep_duration_min','light_sleep_duration_min']].max(axis=1)

In [ ]:
# 요일 컬럼 추가 (예: 월요일, 화요일, ...)
train['lifelog_date'] = pd.to_datetime(train['lifelog_date'])
test['lifelog_date'] = pd.to_datetime(test['lifelog_date'])

# 요일
weekday_map = {
    0: '월요일', 1: '화요일', 2: '수요일', 3: '목요일',
    4: '금요일', 5: '토요일', 6: '일요일'
}
train['weekday'] = train['lifelog_date'].dt.dayofweek.map(weekday_map)
test['weekday'] = test['lifelog_date'].dt.dayofweek.map(weekday_map)

# 월
train['month'] = train['lifelog_date'].dt.month
test['month'] = test['lifelog_date'].dt.month

# weekend
train['weekend'] = np.where(train['weekday'].isin(['토요일','일요일']),1,0)
test['weekend'] = np.where(test['weekday'].isin(['토요일','일요일']),1,0)

# 공휴일
공휴일 = [
     '2024-08-15'
    ,'2024-09-16'
    ,'2024-09-17'
    ,'2024-09-18'
    ,'2024-10-03'
    ,'2024-10-09'
]
train['공휴일'] = np.where(train['lifelog_date'].isin(공휴일),1,0)
test['공휴일'] = np.where(test['lifelog_date'].isin(공휴일),1,0)

# 주말 + 공휴일 묶어주기
# train['weekend'] = np.where( ((train['weekend']==0) & (train['공휴일']==1)), 1, train['weekend'])
# test['weekend'] = np.where( ((test['weekend']==0) & (test['공휴일']==1)), 1, test['weekend'])

In [ ]:
def add_prev_day_flag(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    # 각 subject_id별로 전날 날짜 만들기
    df['prev_day'] = df['lifelog_date'] - pd.Timedelta(days=1)

    # subject_id + 날짜 기준으로 원본 키 구성
    key_set = set(zip(df['subject_id'], df['lifelog_date']))

    # 전날 데이터가 존재하면 1, 없으면 0
    df['has_prev_day_data'] = df[['subject_id', 'prev_day']].apply(
        lambda row: 1 if (row['subject_id'], row['prev_day']) in key_set else 0, axis=1
    )

    return df.drop(columns=['prev_day'])

train = add_prev_day_flag(train)
test = add_prev_day_flag(test)

In [ ]:
# 추정휴가
def rule_based_sum(x):
    rules = (
        # (x['sleep_duration_min'] > (x['avg_sleep_duration']+30))
          (x['sleep_duration_min'] > (x['avg_sleep_duration']+60))
        & (x['week_type'] == 'weekday')
        # & (x['month'].isin([7,8]))
    )
    return rules

train['vacation'] = train.groupby('subject_id').apply(rule_based_sum).reset_index(level=0, drop=True).astype(int)
test['vacation'] = test.groupby('subject_id').apply(rule_based_sum).reset_index(level=0, drop=True).astype(int)

# check
test.groupby(['subject_id'])['vacation'].sum()

,vacation
subject_id,
id01,2
id02,3
id03,4
id04,9
id05,4
id06,9
id07,14
id08,2
id09,5


In [ ]:
# 숫자형 컬럼만 선택해서 결측값 -1로 채우기
train[train.select_dtypes(include='number').columns] = train.select_dtypes(include='number').fillna(-1)
test[test.select_dtypes(include='number').columns] = test.select_dtypes(include='number').fillna(-1)

In [ ]:
def remove_highly_correlated_features(X, threshold=0.95):
    # 상관계수 행렬 계산 (절대값 기준)
    corr_matrix = X.corr().abs()

    # 상삼각행렬만 사용 (자기 자신과의 상관 제외)
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # threshold 초과하는 피처 리스트 확보
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    print(f"제거할 feature 개수: {len(to_drop)} / 전체 feature 개수: {X.shape[1]}")
    return X.drop(columns=to_drop), to_drop

In [ ]:
def get_oof_predictions(X, y, params, n_splits=5, is_multiclass=False, num_class=None, early_stop=False):

    oof_preds = np.zeros(len(X))  # ✅ 1차원으로 변경
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    for train_idx, valid_idx in skf.split(X, y):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        if is_multiclass:
            model = LGBMClassifier(**params, objective='multiclass', num_class=num_class)
        else:
            model = LGBMClassifier(**params)

        if early_stop:
            model.fit(
                X_train, y_train,
                eval_set=[(X_train, y_train), (X_valid, y_valid)],
                callbacks=[early_stopping(stopping_rounds=100, verbose=False)]
            )
        else:
            model.fit(X_train, y_train)

        preds = model.predict(X_valid)  # ✅ returns 1D array
        oof_preds[valid_idx] = preds  # ✅ 1D -> 1D 저장

    return oof_preds

In [111]:
# 주의: 상관관계 기반 피처 상위 50개를 타겟별로 누적하고, 이를 전체 학습 시에도 공통적으로 사용하도록 수정된 run_basemodel

def run_basemodel(train, test, valid_ids, best_param_dict, topn, n_splits=5, random_state=42, early_stop=False):
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import f1_score
    from lightgbm import LGBMClassifier, early_stopping
    from category_encoders import TargetEncoder

    train_df = train.copy()
    test_df = test.copy()

    submission_final = test_df[['subject_id', 'sleep_date', 'lifelog_date']].copy()
    submission_final['lifelog_date'] = pd.to_datetime(submission_final['lifelog_date']).dt.date

    # 타겟
    targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3']
    targets_binary_name = ['기상직후수면질','취침전신체적피로','취침전스트레스','수면효율','수면잠들기시간']
    target_multiclass = 'S1'
    all_targets = targets_binary + [target_multiclass]

    def add_noise(series, noise_level, seed=3):
        rng = np.random.default_rng(seed)
        return series * (1 + noise_level * rng.standard_normal(len(series)))

    noise_level = 0.015

    for tgt in all_targets:
        encoder_feats = ['subject_id','month','weekend']
        subject_mean = train_df.groupby(encoder_feats)[tgt].mean().rename(f'{tgt}_te')
        train_df = train_df.merge(subject_mean, on=encoder_feats, how='left')
        test_df = test_df.merge(subject_mean, on=encoder_feats, how='left')
        global_mean = train_df[tgt].mean()
        test_df[f'{tgt}_te'] = test_df[f'{tgt}_te'].fillna(global_mean)

        train_df[f'{tgt}_te'] = add_noise(train_df[f'{tgt}_te'], noise_level)
        test_df[f'{tgt}_te'] = add_noise(test_df[f'{tgt}_te'], noise_level)

        train_df['TMP'] = train_df[encoder_feats].applymap(str).agg(''.join, axis=1)
        test_df['TMP'] = test_df[encoder_feats].applymap(str).agg(''.join, axis=1)

        encoder = TargetEncoder(cols=['TMP'], smoothing=300)
        encoder.fit(train_df[['TMP']], train_df[tgt])

        train_df[f'{tgt}_te2'] = add_noise(encoder.transform(train_df[['TMP']]).iloc[:, 0], noise_level)
        test_df[f'{tgt}_te2'] = add_noise(encoder.transform(test_df[['TMP']]).iloc[:, 0], noise_level)

        train_df.drop(columns=['TMP'], inplace=True)
        test_df.drop(columns=['TMP'], inplace=True)

    PK = ['sleep_date', 'lifelog_date', 'subject_id']
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object', 'category']).columns if i not in PK+['pk']]
    for col in categorical_features:
        train_df[col] = encoder.fit_transform(train_df[col])
        test_df[col] = encoder.fit_transform(test_df[col])

    X = train_df.drop(columns=PK + all_targets)
    test_X = test_df.drop(columns=PK + all_targets)

    print("# STEP1: 실험 결과 확인")
    print("=============== Validation Results ==============")
    total_avg_f1s = []
    best_iteration_temp = {k: [] for k in all_targets}
    val_f1 = []

    selected_feature_set = set()
    top_features_dict = {}

    for col in targets_binary:

        # 상관관계
        y = train_df[col]
        corr_series = X.corrwith(y).abs()
        top_features = corr_series.sort_values(ascending=False).head(topn).index.tolist()
        selected_feature_set.update(top_features)
        top_features_dict[col] = top_features

        valid_ids['pk'] = valid_ids['subject_id'] + valid_ids['sleep_date']
        train_df['pk'] = train_df['subject_id'] + train_df['sleep_date']

        X_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']), top_features].reset_index(drop=True)
        X_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']), top_features].reset_index(drop=True)
        y_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']), col].reset_index(drop=True)
        y_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']), col].reset_index(drop=True)

        print(f'# 타겟: {col}, 변수개수:{len(top_features)}')

        model = LGBMClassifier(**best_param_dict[col], random_state=random_state)
        if early_stop:
            model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)],
                      callbacks=[early_stopping(500, verbose=False)])
            best_iteration_temp[col].append(model.best_iteration_)
        else:
            model.fit(X_train, y_train)
            best_iteration_temp[col].append(5000)

        pred_valid = model.predict(X_valid)
        f1 = f1_score(y_valid, pred_valid, average='macro')
        val_f1.append(f1)

    # 상관관계
    y = train_df['S1']
    corr_series = X.corrwith(y).abs()
    top_features = corr_series.sort_values(ascending=False).head(topn).index.tolist()
    selected_feature_set.update(top_features)
    top_features_dict['S1'] = top_features

    print(f'# 타겟: S1, 변수개수:{len(top_features)}')

    X_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']), top_features].reset_index(drop=True)
    X_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']), top_features].reset_index(drop=True)
    y_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']), 'S1'].reset_index(drop=True)
    y_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']), 'S1'].reset_index(drop=True)

    model = LGBMClassifier(**best_param_dict['S1'], objective='multiclass', num_class=3, random_state=random_state)
    if early_stop:
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  callbacks=[early_stopping(500, verbose=False)])
        best_iteration_temp['S1'].append(model.best_iteration_)
    else:
        model.fit(X_train, y_train)
        best_iteration_temp['S1'].append(5000)

    pred_valid = model.predict(X_valid)
    f1 = f1_score(y_valid, pred_valid, average='macro')
    val_f1.append(f1)
    avg_f1 = np.mean(val_f1)
    detail = " ".join([f"{name}({tname}):{score:.4f}" for name, tname, score in zip(targets_binary + ['S1'], targets_binary_name + ['S1'], val_f1)])
    print(f"# 평균 F1: {avg_f1:.4f} / [상세] {detail}")

    selected_feature_list = list(selected_feature_set)
    X = X[selected_feature_list]
    test_X = test_X[selected_feature_list]
    print(f"# 전체 학습 피처 수: {len(selected_feature_list)}개")

    # === 전체 재학습 및 예측 ===
    for col in targets_binary:
        y = train_df[col]
        model = LGBMClassifier(**best_param_dict[col], random_state=random_state)
        model.fit(X[top_features_dict[col]], y)
        submission_final[col] = model.predict(test_X[top_features_dict[col]])

        # vi[1]
        fi_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
        top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
        feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
        print(f"[{col}] {feat_str}")

    # S1 예측
    y = train_df['S1']
    model = LGBMClassifier(**best_param_dict['S1'], objective='multiclass', num_class=3, random_state=random_state)
    model.fit(X[top_features_dict['S1']], y)
    submission_final['S1'] = model.predict(test_X[top_features_dict['S1']])

    # vi[2]
    fi_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
    top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
    feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
    print(f"[S1] {feat_str}")

    # 예측 저장
    submission_final = submission_final[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]
    fname = f"/content/drive/MyDrive/data/submission_{avg_f1}.csv"
    submission_final.to_csv(fname, index=False)
    print(f"# {fname} 저장 완료")
    print(f"# submission shape:{submission_final.shape}")
    print("================================================")

    # 모델별 예측결과 비율 비교
    a11 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
    a13 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
    a12 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
    a21 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
    a23 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
    a22 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
    result = pd.concat([a11, a13, a12, a21, a23, a22], axis=1)
    result.columns = ['학습sum','학습len','학습mean','테스트sum','테스트len','테스트mean']
    print('# 예측결과 비교표')
    display(result)

    # === STEP2: OOF 예측 생성 ===
    print(f"# STEP2: OOF 예측 생성")
    oof_result = train_df[['subject_id', 'sleep_date', 'lifelog_date']].copy()
    oof_f1 = []

    for col in targets_binary:
        y = train_df[col]
        corr_series = X.corrwith(y).abs()
        top_features = corr_series.sort_values(ascending=False).head(topn).index.tolist()

        model = LGBMClassifier(**best_param_dict[col], random_state=random_state)
        oof_preds = np.zeros_like(y)

        from sklearn.model_selection import KFold
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        for train_index, valid_index in kf.split(X):
            model.fit(X.iloc[train_index][top_features], y.iloc[train_index])
            oof_preds[valid_index] = model.predict(X.iloc[valid_index][top_features])

        oof_result[col] = oof_preds
        f1 = f1_score(y, oof_preds, average='macro')
        oof_f1.append(f1)
        print(f"# [OOF - {col}] F1 score: {f1:.4f}")

    # S1
    col = 'S1'
    y = train_df[col]
    corr_series = X.corrwith(y).abs()
    top_features = corr_series.sort_values(ascending=False).head(topn).index.tolist()
    model = LGBMClassifier(**best_param_dict[col], objective='multiclass', num_class=3, random_state=random_state)
    oof_preds = np.zeros_like(y)

    for train_index, valid_index in kf.split(X):
        model.fit(X.iloc[train_index][top_features], y.iloc[train_index])
        oof_preds[valid_index] = model.predict(X.iloc[valid_index][top_features])

    oof_result[col] = oof_preds
    f1 = f1_score(y, oof_preds, average='macro')
    oof_f1.append(f1)
    print(f"# [OOF - {col}] F1 score: {f1:.4f}")
    print(f"# [OOF] 평균 F1 score: {np.mean(oof_f1):.4f}")

    return submission_final, oof_result

### 📌 run

In [109]:
drop_features2 = [
  'vehicle_minutes',
  'hr_afternoon_above_100_ratio',
  'light_week_type_lag1',
  'activehour_top_bssid',
  'beforebed_top_bssid'
]

In [110]:
# 공통 하이퍼파라미터
common_params = {
  'n_estimators': 5000,
  "learning_rate": 0.005,
  # "shrinkage_rate": 0.12,
  # 'min_data_in_leaf':2,
  # 'bagging_fraction':0.9,
  # 'feature_fraction':0.6,
  'lambda_l1': 5,
  'lambda_l2': 1,
  # 'max_depth': 20,
  'n_jobs': -1,
  'verbosity': -1
}

common_params.update({
    'boosting_type': 'dart',
    'learning_rate': 0.01,
    'n_estimators': 1500,
    'feature_fraction': 0.6, # 0.6
    'bagging_fraction': 0.8, # 0.8
    'bagging_freq': 1, # 1
    'lambda_l1': 5,    # 5
    'lambda_l2': 1,    # 1
    'drop_rate': 0.1,  # 0.1,
    'skip_drop': 0.5,  # 0.5,
    'max_drop': 50,
    'uniform_drop': False,
    'verbosity': -1,
    'n_jobs': -1
})

# 모델별 세부 하이퍼파라미터
best_param_dict = {}

# 공통 하이퍼파라미터 대체 (이상한 모델의 경우)
best_param_dict['Q3'] = common_params
best_param_dict['S1'] = common_params
best_param_dict['S2'] = common_params
best_param_dict['S3'] = common_params
best_param_dict['Q1'] = common_params
best_param_dict['Q2'] = common_params

# -----------------------
# [vld] F1 score: 0.63590
# [OOF] F1 score: 0.66200
# [pbl] F1 score: 0.62832
# -----------------------

# 전체 평균 F1: 0.6490
# [OOF] F1 score: 0.6708

# GPS 삭제
# 전체 평균 F1: 0.6531
# [OOF] F1 score: 0.6703
"""
# STEP1: 실험 결과 확인
=============== Validation Results ==============
# 평균 F1: 0.6274 / [상세] Q1(기상직후수면질):0.6847 Q2(취침전신체적피로):0.7597 Q3(취침전스트레스):0.6143 S2(수면효율):0.5423 S3(수면잠들기시간):0.6769 S1(S1):0.4865

# STEP2: 전체 학습 피처 수: 158개

# STEP3: OOF 예측 생성
# [OOF - Q1] F1 score: 0.7044
# [OOF - Q2] F1 score: 0.6894
# [OOF - Q3] F1 score: 0.6780
# [OOF - S2] F1 score: 0.6846
# [OOF - S3] F1 score: 0.7150
# [OOF - S1] F1 score: 0.5635
# [OOF] 평균 F1 score: 0.6725
"""
selected_features = [i for i in train.columns if i not in drop_features2]
submission_final, oof_result = run_basemodel(train[selected_features], test[selected_features], valid_ids, best_param_dict, topn=10, n_splits=5, random_state=41, early_stop=False)

# STEP1: 실험 결과 확인
=============== Validation Results ==============
# 타겟: Q1, 변수개수:10
# 타겟: Q2, 변수개수:10
# 타겟: Q3, 변수개수:10
# 타겟: S2, 변수개수:10
# 타겟: S3, 변수개수:10
# 타겟: S1, 변수개수:10
# 평균 F1: 0.6325 / [상세] Q1(기상직후수면질):0.6755 Q2(취침전신체적피로):0.8087 Q3(취침전스트레스):0.5945 S2(수면효율):0.5404 S3(수면잠들기시간):0.6854 S1(S1):0.4908
# 전체 학습 피처 수: 40개
# /content/drive/MyDrive/data/submission_0.6325195826339024.csv 저장 완료
# submission shape:(250, 9)
# 예측결과 비교표


,학습sum,학습len,학습mean,테스트sum,테스트len,테스트mean
Q1,223,450,0.4956,137,250,0.5480
Q2,253,450,0.5622,147,250,0.5880
Q3,270,450,0.6000,176,250,0.7040
S1,390,450,0.8667,186,250,0.7440
S2,293,450,0.6511,165,250,0.6600
S3,298,450,0.6622,171,250,0.6840


# STEP2: OOF 예측 생성
# [OOF - Q1] F1 score: 0.7000
# [OOF - Q2] F1 score: 0.6869
# [OOF - Q3] F1 score: 0.7034
# [OOF - S2] F1 score: 0.6688
# [OOF - S3] F1 score: 0.7119
# [OOF - S1] F1 score: 0.5728
# [OOF] 평균 F1 score: 0.6740


### 📌 feature selection

In [ ]:
%%time

from sklearn.inspection import permutation_importance

def run_feature_selection_by_permutation(train_df, y_col, params, max_remove_ratio=0.5, remove_per_iter=5, random_state=42):
    feature_log = []

    # 전처리용 변수 정의
    PK = ['sleep_date', 'lifelog_date', 'subject_id']
    all_targets = ['Q1', 'Q2', 'Q3', 'S2', 'S3', 'S1']

    # label encoding
    encoder = LabelEncoder()
    cat_cols = train_df.select_dtypes(include=['object', 'category']).columns
    cat_cols = [col for col in cat_cols if col not in PK + ['pk']]
    for col in cat_cols:
        train_df[col] = encoder.fit_transform(train_df[col].astype(str))

    # X, y 정의
    y = train_df[y_col].copy()
    X = train_df.drop(columns=PK + all_targets)
    features = X.columns.tolist()

    initial_n_features = len(features)
    min_features = int(initial_n_features * (1 - max_remove_ratio))
    iter_idx = 0

    total_iters = (initial_n_features - min_features) // remove_per_iter
    pbar = tqdm(total=total_iters, desc="Feature Selection")

    while len(features) > min_features:
        importances = pd.Series(0.0, index=features)
        f1_scores = []

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
        for train_idx, val_idx in tqdm(skf.split(X[features], y), total=5, desc=f"Iteration {iter_idx}", leave=False):
            X_train, X_val = X.iloc[train_idx][features], X.iloc[val_idx][features]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            model = LGBMClassifier(**params, random_state=random_state)
            model.fit(X_train, y_train)

            result = permutation_importance(
                model, X_val, y_val, scoring='f1_macro', n_repeats=3,
                random_state=random_state, n_jobs=-1
            )

            fold_importance = pd.Series(result.importances_mean, index=features)
            importances += fold_importance

            y_pred = model.predict(X_val)
            f1_scores.append(f1_score(y_val, y_pred, average='macro'))

        importances /= skf.n_splits
        worst_features = importances.sort_values().head(remove_per_iter).index.tolist()

        feature_log.append({
            'iteration': iter_idx,
            'mean_f1': round(np.mean(f1_scores), 5),
            'removed_features': worst_features
        })

        features = [f for f in features if f not in worst_features]
        iter_idx += 1
        pbar.update(1)

    pbar.close()
    log_df = pd.DataFrame(feature_log)
    return log_df

In [ ]:
%%time

# log_df = run_feature_selection_by_permutation(train, y_col='S1', params=best_param_dict['S1'])
# display(log_df)

### 📌 실험 [S1:focal loss]

In [ ]:
def get_oof_predictions2(X, y, params, n_splits=5, is_multiclass=False, num_class=None, early_stop=False):
    oof_preds = np.zeros(len(X))
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # object 타입 컬럼 제거 (예: 'pk')
    # X = X.select_dtypes(include=[np.number])
    X = X[[i for i in X.columns if i not in ['pk']]]

    for train_idx, valid_idx in skf.split(X, y):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        if is_multiclass:
            # 앙상블 방식 적용 (0 vs 1+2) -> 1 vs 2
            y_train_bin_0vs12 = (y_train != 0).astype(int)
            model_0vs12 = LGBMClassifier(**params, objective='binary')
            model_0vs12.fit(X_train, y_train_bin_0vs12)
            pred_12 = model_0vs12.predict(X_valid)

            y_train_bin_1vs2 = y_train[y_train != 0].copy()
            X_train_1vs2 = X_train[y_train != 0].copy()
            X_valid_1vs2 = X_valid[pred_12 == 1].copy()
            y_train_bin_1vs2 = (y_train_bin_1vs2 == 2).astype(int)

            model_1vs2 = LGBMClassifier(**params, objective='binary')
            model_1vs2.fit(X_train_1vs2, y_train_bin_1vs2)

            pred_1vs2 = model_1vs2.predict(X_valid_1vs2)

            preds = []
            idx_1vs2 = 0
            for i, p_12 in enumerate(pred_12):
                if p_12 == 0:
                    preds.append(0)
                else:
                    p_2 = pred_1vs2[idx_1vs2]
                    idx_1vs2 += 1
                    preds.append(2 if p_2 == 1 else 1)
        else:
            model = LGBMClassifier(**params)
            if early_stop:
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_train, y_train), (X_valid, y_valid)],
                    callbacks=[early_stopping(stopping_rounds=100, verbose=False)]
                )
            else:
                model.fit(X_train, y_train)
            preds = model.predict(X_valid)

        oof_preds[valid_idx] = preds

    return oof_preds

In [ ]:
def focal_loss_lgb(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        # y_true: (n_samples,)
        # y_pred: (n_samples,) raw score
        a = alpha
        g = gamma

        p = 1 / (1 + np.exp(-y_pred))  # sigmoid

        grad = a * (y_true * (1 - p) ** g * (g * p * np.log(np.clip(p, 1e-9, 1)) + p - 1)
                    + (1 - y_true) * p ** g * (g * (1 - p) * np.log(np.clip(1 - p, 1e-9, 1)) - p))

        hess = a * (y_true * (1 - p) ** (g - 1) * (
                    g * (g + 1) * p * (1 - p) * np.log(np.clip(p, 1e-9, 1)) +
                    (1 - p) * (1 - 2 * p))
                    + (1 - y_true) * p ** (g - 1) * (
                    g * (g + 1) * p * (1 - p) * np.log(np.clip(1 - p, 1e-9, 1)) +
                    p * (2 * p - 1)))

        return grad, hess
    return loss

In [ ]:
def train_lgb_with_focal(X, y, params, num_boost_round=6000):
    dtrain = lgb.Dataset(X, label=y)
    model = lgb.train(
        params={k: v for k, v in params.items() if k != 'objective'},  # learning_rate는 아래에서 설정
        train_set=dtrain,
        feval=focal_loss_lgb(alpha=0.25, gamma=2.0),
        num_boost_round=num_boost_round
    )
    return model

In [ ]:
def predict_S1_with_two_models(X_train, y_train, test_X, params, thred1, thred2):
    # 1) 0 vs (1+2)
    # y_train_bin_0vs12 = (y_train != 0).astype(int)
    # model_0vs12 = LGBMClassifier(**params, objective='binary')
    # model_0vs12.fit(X_train, y_train_bin_0vs12)
    # pred_12 = model_0vs12.predict(test_X)
    y_train_bin_0vs12 = (y_train != 0).astype(int)
    model_0vs12 = train_lgb_with_focal(X_train, y_train_bin_0vs12, params)
    pred_12 = model_0vs12.predict(test_X) > thred1

    # 2) (0+1) vs 2
    # y_train_bin_01vs2 = (y_train == 2).astype(int)
    # model_01vs2 = LGBMClassifier(**params, objective='binary')
    # model_01vs2.fit(X_train, y_train_bin_01vs2)
    # pred_2 = model_01vs2.predict(test_X)
    y_train_bin_01vs2 = (y_train == 2).astype(int)
    model_01vs2 = train_lgb_with_focal(X_train, y_train_bin_01vs2, params)
    pred_2 = model_01vs2.predict(test_X) > thred2

    y_pred = []
    for p_12, p_2 in zip(pred_12, pred_2):
        if p_2:
            y_pred.append(2)
        elif p_12:
            y_pred.append(1)
        else:
            y_pred.append(0)

    return np.array(y_pred)

In [ ]:
def run_basemodel2(train, test, valid_ids, common_params, n_splits, thred1, thred2, random_state=42, early_stop=False):

    train_df = train.copy()
    test_df = test.copy()

    submission_final = test_df[['subject_id', 'sleep_date', 'lifelog_date']].copy()
    submission_final['lifelog_date'] = pd.to_datetime(submission_final['lifelog_date']).dt.date

    # 타겟
    targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3']
    targets_binary_name = ['기상직후수면질','취침전신체적피로','취침전스트레스','수면효율','수면잠들기시간']
    target_multiclass = 'S1'
    all_targets = targets_binary + [target_multiclass]

    # 노이즈 수준 설정
    def add_noise(series, noise_level, seed=3):
        rng = np.random.default_rng(seed)
        return series * (1 + noise_level * rng.standard_normal(len(series)))

    noise_level = 0.015  # 필요에 따라 조정

    # 타겟인코딩
    # m = 0: 스무딩 없이 범주별 평균만 사용합니다. 관측 수가 많은 범주에는 적합하지만, 적은 경우 과적합 위험이 있습니다.
    # m = 1~10: 일반적인 기본값으로, 대부분의 상황에서 안정적인 성능을 보입니다.
    # m = 50~300: 관측 수가 매우 적은 범주가 많거나 데이터가 희소한 경우에 유용합니다.
    for tgt in all_targets:

      encoder_feats = ['subject_id','month','weekend'] # 'weekday', 'subject_id','month','weekend'

      #### 타겟인코딩1

      subject_mean = train_df.groupby(encoder_feats)[tgt].mean().rename(f'{tgt}_te')
      train_df = train_df.merge(subject_mean, on=encoder_feats, how='left')
      test_df = test_df.merge(subject_mean, on=encoder_feats, how='left')
      global_mean = train_df[tgt].mean()
      test_df[f'{tgt}_te'] = test_df[f'{tgt}_te'].fillna(global_mean)

      # 노이즈 추가
      train_df[f'{tgt}_te'] = add_noise(train_df[f'{tgt}_te'], noise_level)
      test_df[f'{tgt}_te'] = add_noise(test_df[f'{tgt}_te'], noise_level)

      #### 타겟인코딩2

      # 새로운 범주형 열 생성
      train_df['TMP'] = train_df[encoder_feats].applymap(str).apply(lambda x: ''.join(x) ,axis=1)
      test_df['TMP'] = test_df[encoder_feats].applymap(str).apply(lambda x: ''.join(x) ,axis=1)

      # 인코더
      encoder = TargetEncoder(cols=['TMP'], smoothing=300) # 40
      encoder.fit(train_df[['TMP']], train_df[tgt])

      # 인코딩 결과를 새로운 열에 저장
      train_df[f'{tgt}_te2'] = encoder.transform(train_df[['TMP']])
      test_df[f'{tgt}_te2'] = encoder.transform(test_df[['TMP']])

      # 노이즈 추가
      train_df[f'{tgt}_te2'] = add_noise(train_df[f'{tgt}_te2'], noise_level)
      test_df[f'{tgt}_te2'] = add_noise(test_df[f'{tgt}_te2'], noise_level)

      # 불필요한 변수 제거
      train_df = train_df.drop(columns=['TMP'])
      test_df = test_df.drop(columns=['TMP'])

    # 인코딩
    PK = ['sleep_date', 'lifelog_date', 'subject_id']
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object', 'category']).columns if i not in PK+['pk']]
    for col in categorical_features:
        print(col)
        train_df[col] = encoder.fit_transform(train_df[col])
        test_df[col] = encoder.fit_transform(test_df[col])

    # Add 'pk' column for splitting
    train_df['pk'] = train_df['subject_id'] + train_df['sleep_date'].astype(str)
    test_df['pk'] = test_df['subject_id'] + test_df['sleep_date'].astype(str)
    valid_ids['pk'] = valid_ids['subject_id'] + valid_ids['sleep_date'].astype(str)

    # X
    # Exclude 'pk' from feature columns
    X = train_df.drop(columns=PK + all_targets + ['pk'])
    test_X = test_df.drop(columns=PK + all_targets + ['pk'])
    print(f'# X shape: {X.shape}')
    print(f'# test_X shape: {test_X.shape}')

    print('\n STEP1: 실험 결과 확인')
    print("=============== Validation Results ==============")
    total_avg_f1s = []
    best_iteration_temp = {k: [] for k in all_targets}

    val_f1 = []
    for col in targets_binary:

        # binary
        y = train_df[col]

        # Use 'pk' for splitting
        X_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
        X_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
        y_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
        y_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
        # X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

        best_param = best_param_dict[col].copy()
        best_param['random_state'] = random_state
        model = LGBMClassifier(**best_param)

        if early_stop:
            model.fit(
                X_train, y_train,
                eval_set=[(X_train, y_train), (X_valid, y_valid)],
                callbacks=[early_stopping(stopping_rounds=100,verbose=False)]
            )
            best_iteration_temp[col].append(model.best_iteration_)
        else:
            model.fit(X_train, y_train)
            best_iteration_temp[col].append(1000)

        pred_valid = model.predict(X_valid)
        f1 = f1_score(y_valid, pred_valid, average='macro')
        val_f1.append(f1)

    # multi
    y = train_df[target_multiclass]

    # Use 'pk' for splitting
    X_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
    X_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
    y_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
    y_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
    # X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    best_param = best_param_dict['S1'].copy()
    best_param['random_state'] = random_state

    # -----------
    pred_valid = predict_S1_with_two_models(X_train, y_train, X_valid, best_param, thred1, thred2)
    # -----------

    f1 = f1_score(y_valid, pred_valid, average='macro')
    val_f1.append(f1)

    avg_f1 = np.mean(val_f1)
    total_avg_f1s.append(avg_f1)
    detail = " ".join([f"{name}({tname}):{score:.4f}" for name, tname, score in zip(targets_binary + [target_multiclass], targets_binary_name + ['S1'], val_f1)])
    print(f" 평균 F1: {avg_f1:.4f} / [상세] {detail}")

    print(f"# 전체 평균 F1: {np.mean(total_avg_f1s):.4f}")
    print("================================================")

    # modoling with 100% train & no valid
    print('\n STEP2: 전체 데이터로 모델 재학습')
    print("====== modoling with 100% train & no valid =====")

    # binary
    binary_preds = {}
    for col in targets_binary:
        binary_params = best_param_dict[col].copy()
        binary_params['random_state'] = random_state
        y = train_df[col]

        model = LGBMClassifier(**binary_params)
        model.fit(X, y)

        binary_preds[col] = model.predict(test_X)
        fi_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
        top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
        feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
        print(f"[{col}] {feat_str}")

    # multiclass
    y = train_df['S1']
    binary_params = best_param_dict['S1'].copy()
    binary_params['random_state'] = random_state

    model = LGBMClassifier(**binary_params)
    model.fit(X, y)

    multiclass_pred = model.predict(test_X)
    fi_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
    top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
    feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
    print(f"[S1] {feat_str}")

    # 예측 저장
    submission_final['S1'] = multiclass_pred
    for col in targets_binary:
      submission_final[col] = binary_preds[col]
    submission_final = submission_final[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]
    fname = f"/content/drive/MyDrive/data/submission_{np.mean(total_avg_f1s)}.csv"
    submission_final.to_csv(fname, index=False)
    print(f"# {fname} 저장 완료")
    print(f"# submission shape:{submission_final.shape}")
    print("================================================")

    # 모델별 예측결과 비율 비교
    a11 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
    a13 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
    a12 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
    a21 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
    a23 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
    a22 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
    result = pd.concat([a11, a13, a12, a21, a23, a22], axis=1)
    result.columns = ['학습sum','학습len','학습mean','테스트sum','테스트len','테스트mean']
    print('\n STEP3: 예측결과 비교표')
    display(result)

    # === STEP4: OOF 예측 생성 (train set에 대해) ===

    # n_splits = 10
    mask = train_df['month'] != 6
    print(f'# k-fold: {n_splits}')
    print(f'# train: {len(y[mask])}')

    oof_f1 = []
    print('\n STEP4: OOF 예측 생성')
    oof_result = train_df[['subject_id', 'sleep_date', 'lifelog_date']].copy()
    for col in targets_binary:
        params = best_param_dict[col].copy()
        params['random_state'] = random_state
        y = train_df[col]
        # Pass X without 'pk'
        oof_preds = get_oof_predictions2(X, y, params, n_splits=n_splits, is_multiclass=False)
        oof_result[col] = oof_preds
        f1 = f1_score(y[mask], oof_preds[mask], average='macro')
        oof_f1.append(f1)
        print(f"[OOF - {col}] F1 score: {f1:.4f}")

    # multiclass
    col = 'S1'
    params = best_param_dict[col].copy()
    params['random_state'] = random_state
    y = train_df[col]
    # Pass X without 'pk'
    oof_preds = get_oof_predictions2(X, y, params, n_splits=n_splits, is_multiclass=True, num_class=3)
    oof_result[col] = oof_preds
    f1 = f1_score(y[mask], oof_preds[mask], average='macro')
    oof_f1.append(f1)
    print(f"[OOF - {col}] F1 score: {f1:.4f}")
    print(f"[OOF] F1 score: {np.mean(oof_f1):.4f}")

    # oof_result 저장
    # Ensure 'pk' is dropped before saving if it was added temporarily
    if 'pk' in oof_result.columns:
        oof_result = oof_result.drop(columns=['pk'])

    fname = f"/content/drive/MyDrive/data/oof_result_{np.mean(total_avg_f1s)}.csv"
    oof_result.to_csv(fname, index=False)
    print(f"# {fname} 저장 완료")

    # Drop 'pk' from train_df and test_df if needed for subsequent steps (optional, depends on notebook flow)
    if 'pk' in train_df.columns:
        train_df = train_df.drop(columns=['pk'])
    if 'pk' in test_df.columns:
        test_df = test_df.drop(columns=['pk'])

    # 클래스 비율 계산
    a1 = train['S1'].value_counts(normalize=True).sort_index().rename('train_ratio')
    a2 = submission_final['S1'].value_counts(normalize=True).sort_index().rename('test_ratio')
    merged_dist = pd.concat([a1, a2], axis=1).fillna(0)
    merged_dist = merged_dist.round(3)
    print("\n # S1 클래스별 비율 (Train vs Test)")
    display(merged_dist)

    return submission_final, oof_result

In [ ]:
# 공통 하이퍼파라미터
common_params = {
  'n_estimators': 5000,
  "learning_rate": 0.01,
  # "shrinkage_rate": 0.12,
  # 'min_data_in_leaf':2,
  # 'bagging_fraction':0.9,
  # 'feature_fraction':0.6,
  'lambda_l1': 5,
  'lambda_l2': 1,
  # 'max_depth': 4,
  'n_jobs': -1,
  'verbosity': -1
}

# common_params.update({
#     'boosting_type': 'dart',
#     'learning_rate': 0.01,
#     'n_estimators': 1000,
#     'feature_fraction': 0.6,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 1,
#     'lambda_l1': 5,
#     'lambda_l2': 1,
#     'drop_rate': 0.1,
#     'skip_drop': 0.5,
#     'max_drop': 50,
#     'uniform_drop': False,
#     'verbosity': -1,
#     'n_jobs': -1
# })

# 공통 하이퍼파라미터 대체 (이상한 모델의 경우)
best_param_dict['Q3'] = common_params
best_param_dict['S1'] = common_params
best_param_dict['S2'] = common_params
best_param_dict['S3'] = common_params
best_param_dict['Q1'] = common_params
best_param_dict['Q2'] = common_params

# [변경내용]
# - 코드 오류 수정 : awake blocks
# - S1 타겟 : focal loss 적용
# 전체 평균 F1: 0.6452
# [OOF] F1 score: 0.6650
submission_final, oof_result = run_basemodel2(train, test, valid_ids, best_param_dict, n_splits=5, thred1=0.65, thred2=0.30, random_state=41)